In [591]:
from langchain_community.document_loaders import PyMuPDFLoader
from pydantic import BaseModel, Field
from langchain_core.output_parsers import PydanticOutputParser, StrOutputParser, BaseOutputParser
from textwrap import dedent

from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
from langchain_core.prompts import ChatPromptTemplate
import httpx
import json
import requests
import pandas as pd
import numpy as np
import re

from langchain_pymupdf4llm import PyMuPDF4LLMLoader, PyMuPDF4LLMParser
import pymupdf4llm
import requests
import uuid
from openai import OpenAI

import time
import os
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv(usecwd=True))


use = 'nhf'

if use == 'hf':
    os.environ["BASE_URL"] = "https://router.huggingface.co/v1"
    os.environ["MODEL_NAME"] ='Qwen/Qwen3-32B'
    os.environ["OPENAI_API_KEY"] = os.getenv("HUGGINGFACE_HUB_TOKEN2")
else:
    os.environ["BASE_URL"] = "https://gigachat.devices.sberbank.ru/api/v1"
    os.environ["MODEL_NAME"] ='GigaChat-Max'
    
    url = "https://ngw.devices.sberbank.ru:9443/api/v2/oauth"
    # Создадим идентификатор UUID (36 знаков)
    rq_uid = str(uuid.uuid4())
    payload={
      'scope': 'GIGACHAT_API_PERS'
    }
    headers = {
      'Content-Type': 'application/x-www-form-urlencoded',
      'Accept': 'application/json',
      'RqUID': rq_uid,
      'Authorization': f'Basic {os.getenv("GIGACHAT_AUTH")}'
    }

    response = requests.request("POST", url, headers=headers, data=payload, verify=False)
    os.environ["OPENAI_API_KEY"] = response.json()['access_token']
    

os.environ['TEMPERATURE'] = '0'




from trialmind.pubmed import pmid2papers, PubmedAPIWrapper, pmid2biocxml, parse_bioc_xml
from trialmind.api import StudyCharacteristicsExtraction, ScreeningCriteriaGeneration,\
                            LiteratureScreening, ScreeningCriteriaCTGeneration,\
                            CTScreening
from trialmind.retrievers import split_text_into_chunks
import extract

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


C:\Users\user\anaconda3\envs\llms\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ngw.devices.sberbank.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [333]:
os.getenv("OPENAI_API_KEY")

'eyJjdHkiOiJqd3QiLCJlbmMiOiJBMjU2Q0JDLUhTNTEyIiwiYWxnIjoiUlNBLU9BRVAtMjU2In0.Kjf6m9hwo1tJVrTkZ2qEZeSFIhM23WmN0muRKYL5Bga6r5fdXQD-8SFaBJae3d4lHD6DsgY9h5T0OVTJxP8LMXrr_jTtF5K5LcyGN1qukI1OrZ9Te75Sqw31wIU7UkBAYwGe5XvWsnMzTszX7vqn-sY1KwM9VK7QEANXussuG8KF0V_AO4U68PcXRvW0rYPVwMQEHBnLbm37fQqEkQt9d_vnQC_YA_Q47V8FDejiahAIEbYsVlrNWBS2GK6G9Iha6VZqX7jtIj3bxTjtpfTdbfASaC_Yaq6wUf8G3N9oaB53PZiwwMGrwgx-vcg0C4h0HIcyF6k6104YcGAcSezlIg.5X41CG_lU0IksZ8bVbkR-w.DTEYT8efM9O9_LduFWBP594WrA8ufAZXzwjBkCpXSFTUPiOgp7MZHICOF8raWSIM0zW2WtlbwQ-SScl18Ivb-Ut78yiNOMxNFJ5EpQiNek8vDJwf7dWXfXAvadoBksH5okfd_UBxvs6OP5qcPHxDgUjDaoHBE0wlNvpo3vwp3VhwixCpR-d3XhsdNRB8ztc4kBLewHO9owAqR4FXdPEnh4GShV6t8oWJ7aN7ZUMxwoSLIzDZAmkB3zEahi9wE2msjOV9IfhOCIO_r_Vblrmg8zzoVjf09SPO2J5O7WlyybtzbCtDWCYouRHb-puIDWakEbxGX6HugiviU44qkku6p-l7W7QJeHe4GMjSfNrk2fr1ybeeZcqrBj3dbxZbL9HVvvl8f36osqVRsQzLpl2NuygvVMymDnCyhx2KYGM4M_-fW3cL781wjzbDJWLaJCiQLC1Xq7VkZpGd3c57BVWHy6y05aH-DHtSniJ6UmEz14agh2CGOkquFRJbPrynpy29JQ7hzoXsBG00sFo8Vn2-fxyOIo9_pd8WaYra_V204xQV6

In [276]:
client = OpenAI(
        base_url=os.getenv("BASE_URL"),
        api_key=os.getenv("OPENAI_API_KEY"),
        http_client=httpx.Client(verify=False)
    )
response = client.chat.completions.create(
        model=os.getenv("MODEL_NAME"),
        messages=[{'role':'user','content':'hey\no_think'}],  # Ensure messages is a list
        temperature=0,
    )
response.choices[0].message.content

'Привет! Чем займёмся?'

In [27]:
Results.model_json_schema()

{'$defs': {'FieldResult': {'properties': {'name': {'description': 'Field name that accurately represents the content of the field based on its description.',
     'maxLength': 25,
     'title': 'Name',
     'type': 'string'},
    'value': {'description': 'Extracted information from the text based on the field description.',
     'maxLength': 50,
     'title': 'Value',
     'type': 'string'},
    'source_id': {'description': 'Cited document IDs.',
     'items': {'type': 'integer'},
     'maxItems': 3,
     'minItems': 1,
     'title': 'Source Id',
     'type': 'array'}},
   'required': ['name', 'value', 'source_id'],
   'title': 'FieldResult',
   'type': 'object'}},
 'properties': {'fieldresult': {'$ref': '#/$defs/FieldResult'}},
 'required': ['fieldresult'],
 'title': 'Results',
 'type': 'object'}

In [30]:
Results.model_json_schema()
completion = client.chat.completions.create(
    model=os.getenv("MODEL_NAME"),
    messages=[
        {
            "role": "user",
            "content": f"Generate some examples with two fieldresult with the following JSON schema: {Results.model_json_schema()}  /no_think"
        }
    ],
    response_format={
        "type": "json_object",
        "json_schema": {
            "name": "car-description",
            "schema": Results.model_json_schema()
        },
    },
    temperature=0,
)
print(completion.choices[0].message.content)

### Example JSON data using the provided schema:

```json
{
  "fieldresult": [
    {
      "name": "Author",
      "value": "Иванов Иван Иванович",
      "source_id": [12345]
    },
    {
      "name": "Publication Date",
      "value": "1 января 2023 года",
      "source_id": [67890]
    }
  ]
}
```

#### Explanation:
- **`name`:** The descriptive label for the extracted field (e.g., “Author” or “Publication Date”).
- **`value`:** The actual value extracted from the source text (e.g., a person’s name or date).
- **`source_id`:** An array of integers representing the cited documents where this field was found.

### Additional Examples

1. **Example 1:**
   ```json
   {
     "fieldresult": [
       {
         "name": "ISBN",
         "value": "978-123456789X",
         "source_id": [101, 102]
       },
       {
         "name": "Publisher",
         "value": "ООО \"Издательство Книги и Статьи\"",
         "source_id": [103]
       }
     ]
   }
   ```

2. **Example 2:**
   ```json
   {


### client.responses дает 403 ошибку, т.е. structured output не работает

In [43]:
from openai import OpenAI
from pydantic import BaseModel

client = OpenAI(
    base_url=os.getenv("BASE_URL"),
    api_key=os.getenv("OPENAI_API_KEY"),
    http_client=httpx.Client(http2=True, verify=False)
)

class ResearchPaperExtraction(BaseModel):
    title: str
    authors: list[str]
    abstract: str
    keywords: list[str]

response = client.responses.parse(
    model=os.getenv("MODEL_NAME"),
    input=[
        {
            "role": "system",
            "content": "You are an expert at structured data extraction. You will be given unstructured text from a research paper and should convert it into the given structure.",
        },
        {"role": "user", "content": "..."},
    ],
    text_format=ResearchPaperExtraction,
)

research_paper = response.output_parsed

PermissionDeniedError: <html>
<head><title>403 Forbidden</title></head>
<body>
<center><h1>403 Forbidden</h1></center>
<hr><center>SynGX</center>
</body>
</html>

In [46]:
response = client.responses.create(
    model=os.getenv("MODEL_NAME"),
    input=[{"role": "system", "content": "You are a helpful math tutor. Guide the user through the solution step by step."},
    {"role": "user", "content": "how can I solve 8x + 7 = -23"}],
    text={
    "format": {
        "type": "json_schema",
        "name": "math_response",
        "schema": {
            "type": "object",
            "properties": {
                "steps": {
                    "type": "array",
                    "items": {
                        "type": "object",
                        "properties": {
                            "explanation": {"type": "string"},
                            "output": {"type": "string"}
                        },
                        "required": ["explanation", "output"],
                        "additionalProperties": False
                    }
                },
                "final_answer": {"type": "string"}
            },
            "required": ["steps", "final_answer"],
            "additionalProperties": False
        },
        "strict": False
    }
    }
    )

print(response.output_text)

PermissionDeniedError: <html>
<head><title>403 Forbidden</title></head>
<body>
<center><h1>403 Forbidden</h1></center>
<hr><center>SynGX</center>
</body>
</html>

In [57]:
response = client.chat.completions.create(
        model=os.getenv("MODEL_NAME"),
        messages=[
            {"role": "system", "content": "You are a helpful assistant designed to output JSON."},
            {"role": "user", "content": "Who won the world series in 2020? Please respond in the format {winner: ...}"}
        ],
        text={"format": {"type": "json_object"}}
    )
print(response.choices[0].message.content)

TypeError: Completions.create() got an unexpected keyword argument 'text'

### response_format: {type: "json_schema",...} -- Error code: 400 - {'status': 400, 'message': 'Empty schema with json format type is not supported'}
Structured Outputs with response_format: {type: "json_schema", ...} is only supported with the gpt-4o-mini, gpt-4o-mini-2024-07-18, and gpt-4o-2024-08-06 model snapshots and later.

In [90]:
class CarType(str, Enum):
    sedan = "sedan"
    suv = "SUV"
    truck = "Truck"
    coupe = "Coupe"

class CarDescription(BaseModel):
    brand: str
    model: str
    car_type: CarType

json_schema = CarDescription.model_json_schema()

completion = client.chat.completions.create(
    model=os.getenv("MODEL_NAME"),
    messages=[
        {
            "role": "user",
            "content": "Generate a JSON with the brand, model and car_type of Ford Explorer (SUV) car from the 90's"
        }
    ],
    response_format={
        "type": "json_schema",
        "json_schema": {
            "name": "car-description",
            "schema": CarDescription.model_json_schema()
        },
    },
)
print(completion.choices[0].message.content)

BadRequestError: Error code: 400 - {'status': 400, 'message': 'Empty schema with json format type is not supported'}

### back

In [43]:
from enum import Enum
class CarType(str, Enum):
    sedan = "sedan"
    suv = "SUV"
    truck = "Truck"
    coupe = "Coupe"

class CarDescription(BaseModel):
    brand: str
    model: str
    car_type: CarType

json_schema = CarDescription.model_json_schema()

completion = client.chat.completions.create(
    model=os.getenv("MODEL_NAME"),
    messages=[
        {
            "role": "user",
            "content": "Generate a JSON with the brand, model and car_type of Mazda cars from the 90's  /no_think"
        }
    ],
    response_format={
        "type": "json_object",
        "json_schema": {
            "name": "car-description",
            "schema": CarDescription.model_json_schema()
        },
    },
    temperature=0,
)
print(completion.choices[0].message.content)

### JSON data for Mazda cars from the 1990s:

```json
[
  {
    "brand": "Mazda",
    "model": "MX-5 Miata",
    "car_type": "Roadster"
  },
  {
    "brand": "Mazda",
    "model": "Protege",
    "car_type": "Compact Sedan"
  },
  {
    "brand": "Mazda",
    "model": "Millenia",
    "car_type": "Full-size Luxury Sedan"
  },
  {
    "brand": "Mazda",
    "model": "B2300",
    "car_type": "Pickup Truck"
  },
  {
    "brand": "Mazda",
    "model": "MPV",
    "car_type": "Minivan"
  },
  {
    "brand": "Mazda",
    "model": "626",
    "car_type": "Mid-size Sedan"
  }
]
```

This JSON contains information about popular Mazda models from the 1990s, including their respective brands, models, and types (sedans, roadsters, minivans, etc.).


In [ ]:
response.msgs[0].parsed

In [36]:
'''
Gigachat --  Error code: 400 - {'status': 400, 'message': 'Empty schema with json format type is not supported'}
works for qwen! 
'''
from enum import Enum
class CarType(str, Enum):
    sedan = "sedan"
    suv = "SUV"
    truck = "Truck"
    coupe = "Coupe"

class CarDescription(BaseModel):
    brand: str
    model: str
    car_type: CarType

json_schema = CarDescription.model_json_schema()

completion = client.chat.completions.parse(
    model=os.getenv("MODEL_NAME"),
    messages=[
        {
            "role": "user",
            "content": "Generate a JSON with the brand, model and car_type of Mazda cars from the 90's  /no_think"
        }
    ],
    response_format=CarDescription,
    temperature=0,
)
print(completion.choices[0].message.content)

BadRequestError: Error code: 400 - {'status': 400, 'message': 'Empty schema with json format type is not supported'}

In [37]:
json_schema

{'$defs': {'CarType': {'enum': ['sedan', 'SUV', 'Truck', 'Coupe'],
   'title': 'CarType',
   'type': 'string'}},
 'properties': {'brand': {'title': 'Brand', 'type': 'string'},
  'model': {'title': 'Model', 'type': 'string'},
  'car_type': {'$ref': '#/$defs/CarType'}},
 'required': ['brand', 'model', 'car_type'],
 'title': 'CarDescription',
 'type': 'object'}

In [40]:
from enum import Enum
class CarType(str, Enum):
    sedan = "sedan"
    suv = "SUV"
    truck = "Truck"
    coupe = "Coupe"

class CarDescription(BaseModel):
    brand: str
    model: str
    car_type: CarType

json_schema = CarDescription.model_json_schema()

completion = client.chat.completions.create(
    model=os.getenv("MODEL_NAME"),
    messages=[
        {
            "role": "user",
            "content": "Generate a JSON with the brand, model and car_type of Mazda cars from the 90's"+\
        f" and follow the JSON format defined by the following JSON schema: {json_schema} /no_think"
        }
    ],
    #tools=[json_schema],
    temperature=0,
)
print(completion.choices[0].message.content)

### JSON data for Mazda cars from the 1990s

```json
[
  {
    "brand": "Mazda",
    "model": "626",
    "car_type": "sedan"
  },
  {
    "brand": "Mazda",
    "model": "MX-5",
    "car_type": "Coupe"
  },
  {
    "brand": "Mazda",
    "model": "Bongo",
    "car_type": "Van"
  },
  {
    "brand": "Mazda",
    "model": "MPV",
    "car_type": "Minivan"
  },
  {
    "brand": "Mazda",
    "model": "Navajo",
    "car_type": "Pickup Truck"
  },
  {
    "brand": "Mazda",
    "model": "Protege",
    "car_type": "Hatchback"
  },
  {
    "brand": "Mazda",
    "model": "Millenia",
    "car_type": "Sedan"
  },
  {
    "brand": "Mazda",
    "model": "Tribute",
    "car_type": "SUV"
  }
]
```

### Explanation:
- The `'$defs'` section defines the allowed values for the `car_type`.
- Each entry in the array represents a different Mazda car from the 1990s.
- The required fields (`brand`, `model`, and `car_type`) are specified as per the given JSON schema.
- The `'$ref'` is used to reference the predefi

In [39]:
'''
qwen doesnt support "structured_outputs" in body
'''
class CarType(str, Enum):
    sedan = "sedan"
    suv = "SUV"
    truck = "Truck"
    coupe = "Coupe"

class CarDescription(BaseModel):
    brand: str
    model: str
    car_type: CarType

json_schema = CarDescription.model_json_schema()

completion = client.chat.completions.create(
    model=os.getenv("MODEL_NAME"),
    messages=[
        {
            "role": "user",
            "content": "Generate a JSON with the brand, model and car_type of Mazda cars from the 90's"
        }
    ],
    extra_body={"structured_outputs": {"json": json_schema}}
)
print(completion.choices[0].message.content)

### Example JSON structure for Mazda cars from the 1990s:

```json
[
  {
    "brand": "Mazda",
    "model": "MX-5 Miata",
    "car_type": "Convertible"
  },
  {
    "brand": "Mazda",
    "model": "626",
    "car_type": "Sedan/SUV"
  },
  {
    "brand": "Mazda",
    "model": "Capella",
    "car_type": "Compact Car"
  },
  {
    "brand": "Mazda",
    "model": "Bongo Freestyle",
    "car_type": "Minivan/MPV"
  },
  {
    "brand": "Mazda",
    "model": "Millenia",
    "car_type": "Mid-size Luxury Sedan"
  },
  {
    "brand": "Mazda",
    "model": "Protégé",
    "car_type": "Compact Car"
  }
]
```

#### Explanation:
- The `brand` field contains the manufacturer name (`Mazda`).
- The `model` field specifies the individual car models that were popular in the 1990s.
- The `car_type` indicates the type (sedan, convertible, minivan, etc.) to provide more context about each vehicle.

You can modify or extend this list based on specific Mazda models you want to include from the 1990s.


In [42]:
'''
qwen doesnt support "structured_outputs" in body
'''
class CarType(str, Enum):
    sedan = "sedan"
    suv = "SUV"
    truck = "Truck"
    coupe = "Coupe"

class CarDescription(BaseModel):
    brand: str
    model: str
    car_type: CarType

json_schema = CarDescription.model_json_schema()

completion = client.chat.completions.create(
    model=os.getenv("MODEL_NAME"),
    messages=[
        {
            "role": "user",
            "content": "Generate a JSON with the brand, model and car_type of Mazda cars from the 90's"
        }
    ],
    tools = [json_schema]
)
print(completion.choices[0].message.content)

### JSON format for Mazda Cars from the 1990s

```json
[
  {
    "brand": "Mazda",
    "model": "MX-5 (Miata)",
    "car_type": "Convertible sports car"
  },
  {
    "brand": "Mazda",
    "model": "Protégé",
    "car_type": "Compact sedan/coupe"
  },
  {
    "brand": "Mazda",
    "model": "B-series pickup truck",
    "car_type": "Pickup truck"
  },
  {
    "brand": "Mazda",
    "model": "Millenia",
    "car_type": "Full-size luxury sedan"
  },
  {
    "brand": "Mazda",
    "model": "MPV",
    "car_type": "Minivan"
  },
  {
    "brand": "Mazda",
    "model": "Navajo",
    "car_type": "Sport utility vehicle (SUV)"
  }
]
```

This JSON contains information about popular Mazda models from the 1990s, including their brand, model name, and car type.


## 1. clinical trials

In [3]:
file_path = "docs/CC_215_L00_DL_fusion_fixed_oncobox_ru.pdf"
fin_condition, treatements_eng = extract.info_from_doc(file_path)
fin_condition, treatements_eng

('Colorectal cancer',
 ['Crizotinib',
  'Lorlatinib',
  'Necitumumab',
  'Nimotuzumab',
  'Panitumumab',
  'Cetuximab',
  'Brigatinib',
  'Ramucirumab',
  'Pomalidomide',
  'Tamoxifen',
  'Denosumab',
  'Duvelisib',
  'Inavilisib',
  'Bevacizumab',
  'Anastrozole',
  'Exemestane'])

1. загрузить док
2. взять 1-ую страницу; рег.выражениями взять диагноз
-- с ЛЛМ перевести на англ.
3. найти страницу с таблицей; или взять таблицы с PyMuPDFLoader
4. извлечь таблицу

In [4]:
all_studies = extract.get_clinicaltrials(f'''"{fin_condition}"''', 
                                      #' OR '.join(treatements_eng), 
                                       treatements_eng[0],  
                                      max_studies=10)
all_studies

,hasResults,nctId,briefTitle,overallStatus,briefSummary,conditions,studyType,phases,interventions,outcomeMeasures
0,False,NCT03792568,ALK Inhibitor in Metastatic Colorectal Cancer ...,UNKNOWN,This clinical trial aims to evaluate the effic...,[Colorectal Cancer],INTERVENTIONAL,[NA],[],NaN
1,False,NCT02568267,Basket Study of Entrectinib (RXDX-101) for the...,ACTIVE_NOT_RECRUITING,"This is an open-label, multicenter, global Pha...","[Breast Cancer, Cholangiocarcinoma, Colorectal...",INTERVENTIONAL,[PHASE2],[entrectinib],NaN
2,True,NCT02510001,MEK and MET Inhibition in Colorectal Cancer,COMPLETED,This trial is designed to try two new cancer d...,"[Solid Tumor, Colorectal Cancer]",INTERVENTIONAL,[PHASE1],"[pf-02341066, pd-0325901, binimetinib]","[{'type': 'PRIMARY', 'title': 'Maximal Tolerat..."
3,False,NCT03947385,Study of IDE196 in Patients With Solid Tumors ...,RECRUITING,"This is a Phase 1/2, multi-center, open-label ...","[Metastatic Uveal Melanoma, Cutaneous Melanoma...",INTERVENTIONAL,"[PHASE1, PHASE2]","[ide196, binimetinib, crizotinib]",NaN
4,False,NCT06214793,Taletrectinib in Previously Treated Metastatic...,SUSPENDED,The purpose of this study is to evaluate the s...,"[Breast Cancer, Metastatic Breast Cancer]",INTERVENTIONAL,[PHASE2],[taletrectinib],NaN
5,False,NCT05725200,Study to Investigate Outcome of Individualized...,RECRUITING,The purpose of the study is to investigate the...,[Metastatic Colorectal Cancer],INTERVENTIONAL,[PHASE2],"[alectinib, cetuximab, crizotinib, dasatinib, ...",NaN
6,False,NCT02465060,Targeted Therapy Directed by Genetic Testing i...,ACTIVE_NOT_RECRUITING,This phase II MATCH screening and multi-sub-tr...,"[Advanced Lymphoma, Advanced Malignant Solid N...",INTERVENTIONAL,[PHASE2],"[adavosertib, afatinib, afatinib dimaleate, bi...",NaN


In [13]:
# test the apis

api = ScreeningCriteriaCTGeneration()
ecs = api.run(
    population = f"Patients with {fin_condition} undergoing treatment with {treatements_eng[0]}",
    intervention = f"{treatements_eng[0]}",
    comparator = "",
    outcome = "",
    llm = os.getenv("MODEL_NAME"),
    num_title_criteria=2,
    num_abstract_criteria=2,
    thinking=False,
)
ecs

call_openai https://gigachat.devices.sberbank.ru/api/v1
api_call_single https://gigachat.devices.sberbank.ru/api/v1/
```json
{
    "ELIGIBILITY_ANALYSIS": [
        "Patients must have colorectal cancer confirmed by histopathology or imaging studies",
        "Treatment with crizotinib must be administered as part of standard therapy or experimental protocol",
        "Comparative intervention must involve another drug or placebo",
        "Outcomes must measure efficacy, safety, or survival endpoints relevant to colorectal cancer treatment",
        "Study design must be randomized controlled trial (RCT), cohort study, or case-control study"
    ],
    "TITLE_CRITERIA": [
        "Does the title mention 'colorectal cancer'?",
        "Does the title mention 'crizotinib'?"
    ],
    "CONTENT_CRITERIA": [
        "Is there evidence of randomization or control group in the abstract?",
        "Are outcomes related to tumor response, progression-free survival, overall survival, or advers

{'title_criteria': ["Does the title mention 'colorectal cancer'?",
  "Does the title mention 'crizotinib'?"],
 'content_criteria': ['Is there evidence of randomization or control group in the abstract?',
  'Are outcomes related to tumor response, progression-free survival, overall survival, or adverse events mentioned in the abstract?'],
 'eligibility_analysis': ['Patients must have colorectal cancer confirmed by histopathology or imaging studies',
  'Treatment with crizotinib must be administered as part of standard therapy or experimental protocol',
  'Comparative intervention must involve another drug or placebo',
  'Outcomes must measure efficacy, safety, or survival endpoints relevant to colorectal cancer treatment',
  'Study design must be randomized controlled trial (RCT), cohort study, or case-control study']}

### __check

In [7]:
from trialmind.api import parse_json_outputs

In [29]:
parse_json_outputs(['''```json
{
    "ELIGIBILITY_ANALYSIS": [
        "Patients must have colorectal cancer confirmed by histopathology or imaging studies",
        "Treatment with crizotinib as part of standard therapy or experimental protocol",
        "Comparative intervention against placebo or other treatments",
        "Clearly reported outcomes related to efficacy or safety endpoints",
        "Published in peer-reviewed journals"
    ],
    "TITLE_CRITERIA": [
        "Does the title mention 'colorectal cancer'?",
        "Does the title mention 'crizotinib'?"
    ],
    "CONTENT_CRITERIA": [
        "Is there a description of patient population including diagnosis and stage of disease?",
        "Are details provided about the intervention and comparison groups?"
]
} ```
'''])

[{}]

In [15]:
json_pattern = r"\`\`\`json([\s\S]*?)\`\`\`"
if re.search(r"\`\`\`json([\s\S]*?)", '''```json
{{
    "ELIGIBILITY_ANALYSIS": [
        "Patients must have colorectal cancer confirmed by histopathology or imaging studies",
        "Treatment with crizotinib as part of standard therapy or experimental protocol",
        "Comparative intervention against placebo or other treatments",
        "Clearly reported outcomes related to efficacy or safety endpoints",
        "Published in peer-reviewed journals"
    ],
    "TITLE_CRITERIA": [
        "Does the title mention 'colorectal cancer'?",
        "Does the title mention 'crizotinib'?"
    ],
    "CONTENT_CRITERIA": [
        "Is there a description of patient population including diagnosis and stage of disease?",
        "Are details provided about the intervention and comparison groups?"
]'''):
    print(1)

In [128]:
from typing import Literal
class ScreeningResult(str, Enum):
    evaluations: Literal['YES', 'NO', 'UNCERTAIN'] = Field(description=f"Evaluation for a criteria")
    rationale: str = Field(description="A rationale for a criteria evaluation")

class ScreeningResults(BaseModel):
    one_result: ScreeningResult

json_schema = ScreeningResults.model_json_schema()
json_schema

{'$defs': {'ScreeningResult': {'enum': ["annotation=NoneType required=True description='Evaluation for a criteria'",
    "annotation=NoneType required=True description='A rationale for a criteria evaluation'"],
   'title': 'ScreeningResult',
   'type': 'string'}},
 'properties': {'one_result': {'$ref': '#/$defs/ScreeningResult'}},
 'required': ['one_result'],
 'title': 'ScreeningResults',
 'type': 'object'}

### 2. CT screen

In [14]:
all_text = all_studies.briefTitle.fillna("") + ": " + all_studies.briefSummary.fillna("")
all_text.values.tolist()[:1]

['ALK Inhibitor in Metastatic Colorectal Cancer With ALK Mutation.: This clinical trial aims to evaluate the efficacy, safety of ALK inhibitor in Metastatic Colorectal Cancer Patients with ALK mutation.']

In [22]:

api = CTScreening()
ec_pred = api.run(
    population = f"Patients with {fin_condition} undergoing treatment with {treatements_eng[0]}",
    intervention = f"{treatements_eng[0]}",
    comparator = "",
    outcome = "",
    llm = os.getenv("MODEL_NAME"),
    criteria = ecs['title_criteria'] + ecs['content_criteria'],
    papers = all_text.values.tolist(), # make for the top-100 for demo
)
ec_pred


# CONTEXT #
You are a clinical specialist tasked with assessing clinical trials for inclusion in a meta-analysis based on specific eligibility criteria.

# OBJECTIVE #
Evaluate each criterion of a given clinical trial to determine its eligibility for inclusion in the meta-analysis. Provide a list of decisions ("YES", "NO", or "UNCERTAIN") for each eligibility criterion. You must deliver exactly {num_criteria} responses.

# IMPORTANT NOTE #
If the information within the provided clinical trial content is insufficient to conclusively evaluate a criterion, you must opt for "UNCERTAIN" as your response. Avoid making assumptions or extrapolating beyond the provided data, as accurate and reliable responses are crucial, and fabricating information (hallucinations) could lead to serious errors in the meta-analysis.

# PICO FRAMEWORK #
- P (Patient, Problem or Population): {P}
- I (Intervention): {I}
- C (Comparison): {C}
- O (Outcome): {O}

# PAPER DETAILS #
- Provided clinical trial: {paper_

[{'evaluations': ['YES', 'YES', 'UNCERTAIN', 'YES'],
  'rationale': ["The title mentions 'Colorectal cancer'",
   "The title mentions 'Crizotinib'",
   'Randomization and control group are not explicitly stated in the provided abstract',
   'Outcomes such as tumor response and adverse events are mentioned']},
 {'evaluations': ['NO', 'NO', 'UNCERTAIN', 'NO'],
  'rationale': ['Title does not mention colorectal cancer',
   'Title does not mention crizotinib',
   'Randomization and control group not described in provided details',
   'No outcomes related to tumor response, progression-free survival, overall survival, or adverse events mentioned']},
 {'evaluations': ['YES', 'NO', 'UNCERTAIN', 'YES'],
  'rationale': ['Title mentions colorectal cancer',
   'Title does not mention crizotinib',
   'Randomization or control group not evident from provided info',
   'Outcomes related to tumor response and adverse events mentioned']},
 {'evaluations': ['NO', 'YES', 'UNCERTAIN', 'UNCERTAIN'],
  'ra

In [23]:
evals = [i['evaluations'] for i in ec_pred]

word2int = {"YES": 1, 
            "UNCERTAIN": 0,
            "NO": -1
           }
#rev_subs = { v:k for k, v in word2int.items()} # subs.iteritems() In Python 3
new_evals = []
for one_e in evals:
    new_evals.append([word2int.get(item, 0) for item in one_e ])
new_evals = np.array(new_evals)    
print(new_evals)
print(new_evals.sum(axis=1))
all_studies['screen_eval'] = new_evals.sum(axis=1)
all_studies

[[ 1  1  0  1]
 [-1 -1  0 -1]
 [ 1 -1  0  1]
 [-1  1  0  0]
 [-1 -1  0 -1]
 [ 1  1  0  1]
 [-1  1  0  0]]
[ 3 -3  1  0 -3  3  0]


,hasResults,nctId,briefTitle,overallStatus,briefSummary,conditions,studyType,phases,interventions,outcomeMeasures,screen_eval
0,False,NCT03792568,ALK Inhibitor in Metastatic Colorectal Cancer ...,UNKNOWN,This clinical trial aims to evaluate the effic...,[Colorectal Cancer],INTERVENTIONAL,[NA],[],NaN,3
1,False,NCT02568267,Basket Study of Entrectinib (RXDX-101) for the...,ACTIVE_NOT_RECRUITING,"This is an open-label, multicenter, global Pha...","[Breast Cancer, Cholangiocarcinoma, Colorectal...",INTERVENTIONAL,[PHASE2],[entrectinib],NaN,-3
2,True,NCT02510001,MEK and MET Inhibition in Colorectal Cancer,COMPLETED,This trial is designed to try two new cancer d...,"[Solid Tumor, Colorectal Cancer]",INTERVENTIONAL,[PHASE1],"[pf-02341066, pd-0325901, binimetinib]","[{'type': 'PRIMARY', 'title': 'Maximal Tolerat...",1
3,False,NCT03947385,Study of IDE196 in Patients With Solid Tumors ...,RECRUITING,"This is a Phase 1/2, multi-center, open-label ...","[Metastatic Uveal Melanoma, Cutaneous Melanoma...",INTERVENTIONAL,"[PHASE1, PHASE2]","[ide196, binimetinib, crizotinib]",NaN,0
4,False,NCT06214793,Taletrectinib in Previously Treated Metastatic...,SUSPENDED,The purpose of this study is to evaluate the s...,"[Breast Cancer, Metastatic Breast Cancer]",INTERVENTIONAL,[PHASE2],[taletrectinib],NaN,-3
5,False,NCT05725200,Study to Investigate Outcome of Individualized...,RECRUITING,The purpose of the study is to investigate the...,[Metastatic Colorectal Cancer],INTERVENTIONAL,[PHASE2],"[alectinib, cetuximab, crizotinib, dasatinib, ...",NaN,3
6,False,NCT02465060,Targeted Therapy Directed by Genetic Testing i...,ACTIVE_NOT_RECRUITING,This phase II MATCH screening and multi-sub-tr...,"[Advanced Lymphoma, Advanced Malignant Solid N...",INTERVENTIONAL,[PHASE2],"[adavosertib, afatinib, afatinib dimaleate, bi...",NaN,0


In [24]:
all_studies[all_studies.screen_eval>=0]

,hasResults,nctId,briefTitle,overallStatus,briefSummary,conditions,studyType,phases,interventions,outcomeMeasures,screen_eval
0,False,NCT03792568,ALK Inhibitor in Metastatic Colorectal Cancer ...,UNKNOWN,This clinical trial aims to evaluate the effic...,[Colorectal Cancer],INTERVENTIONAL,[NA],[],NaN,3
2,True,NCT02510001,MEK and MET Inhibition in Colorectal Cancer,COMPLETED,This trial is designed to try two new cancer d...,"[Solid Tumor, Colorectal Cancer]",INTERVENTIONAL,[PHASE1],"[pf-02341066, pd-0325901, binimetinib]","[{'type': 'PRIMARY', 'title': 'Maximal Tolerat...",1
3,False,NCT03947385,Study of IDE196 in Patients With Solid Tumors ...,RECRUITING,"This is a Phase 1/2, multi-center, open-label ...","[Metastatic Uveal Melanoma, Cutaneous Melanoma...",INTERVENTIONAL,"[PHASE1, PHASE2]","[ide196, binimetinib, crizotinib]",NaN,0
5,False,NCT05725200,Study to Investigate Outcome of Individualized...,RECRUITING,The purpose of the study is to investigate the...,[Metastatic Colorectal Cancer],INTERVENTIONAL,[PHASE2],"[alectinib, cetuximab, crizotinib, dasatinib, ...",NaN,3
6,False,NCT02465060,Targeted Therapy Directed by Genetic Testing i...,ACTIVE_NOT_RECRUITING,This phase II MATCH screening and multi-sub-tr...,"[Advanced Lymphoma, Advanced Malignant Solid N...",INTERVENTIONAL,[PHASE2],"[adavosertib, afatinib, afatinib dimaleate, bi...",NaN,0


### 3. CT outcome results
Взяли все результаты, отдельно сохранили испытания с результатами, и з них отбираем рез-ты только с unit==participants

filter by interventional study type?

In [306]:
from trialmind.llm import batch_call_llm

In [307]:
openai_client = OpenAI(
    base_url=os.getenv("BASE_URL"),
    api_key=os.getenv("OPENAI_API_KEY"),
    http_client=httpx.Client(verify=False)
)

In [308]:
os.getenv("BASE_URL")

'https://gigachat.devices.sberbank.ru/api/v1'

In [309]:
chosen

,hasResults,nctId,briefTitle,overallStatus,briefSummary,conditions,studyType,phases,interventions,outcomeMeasures,screen_eval,screening,res_with_part
2,True,NCT02510001,MEK and MET Inhibition in Colorectal Cancer,COMPLETED,This trial is designed to try two new cancer d...,"[Solid Tumor, Colorectal Cancer]",INTERVENTIONAL,[PHASE1],"[pf-02341066, pd-0325901, binimetinib]","[{'type': 'PRIMARY', 'title': 'Maximal Tolerat...",1,"{'evaluations': ['YES', 'NO', 'UNCERTAIN', 'YE...","[{'type': 'PRIMARY', 'title': 'Clinical Respon..."


In [314]:
messages[0]

[{'role': 'system',
  'content': 'You are a helpful medical assistant. Answer with a concise description of the efficiency statistics for EACH of 1 provided outcome measures /no_think'},
 {'role': 'user',
  'content': "[{'type': 'PRIMARY', 'title': 'Clinical Response to Binimetinib Combined With PF-02341066', 'description': 'To investigate response to treatment with RPII dose of Binimetinib with Crizotinib (PF-02341066), in patients with a) RASMT CRC or b) RASWT/cMET mut amplified CRC or c) RASWT/c-MET over-expressed CRC, as defined by stable, partially or completely responding disease, per Response Evaluation Criteria In Solid Tumors Criteria (RECIST v1.1) for target lesions and assessed by CT: Complete Response (CR), Disappearance of all target lesions; Partial Response (PR), \\\\>=30% decrease in the sum of the longest diameter of target lesions; Stable Disease (SD), Neither sufficient shrinkage to qualify for PR nor sufficient increase (\\\\>=20%) to qualify for Progressive Disease

In [523]:
PROMPT_RES_EXTRACTION  = '''
You are a clinical specialist analyzing clinical trial study reports. 
Your task is to to extract specific information as structured data.

# Reply Format: 
Return the information in the following JSON-format.
```json
{{        
    [
        {{
            "population": n,
            "time_frame": "time_frame",
            "outcomes":
                [
                    {{
                        "category_name": "category1",
                        "outcome": k1
                    }},
                    {{
                        "category_name": "category2",
                        "outcome": k2
                    }},
                    ...
                ]
         }},
        ...
    ]
}}
```
You MUST return ONLY valid JSON, Do NOT include any explanations, comments, or extra text.
"""
'''

In [520]:
class Outcome(BaseModel):
    category_name: str = Field(description='Short description of a category')
    outcome: int = Field(description='Percent of participants')

class ClinicalResult(BaseModel):
    population: int = Field(description='Total number of participants.')
    time_frame: str = Field(description='Time frame')
    outcomes: list[Outcome]



In [521]:
ClinicalResult.model_json_schema()

{'$defs': {'Outcome': {'properties': {'category_name': {'description': 'Short description of a category',
     'title': 'Category Name',
     'type': 'string'},
    'outcome': {'description': 'Percent of participants',
     'title': 'Outcome',
     'type': 'integer'}},
   'required': ['category_name', 'outcome'],
   'title': 'Outcome',
   'type': 'object'}},
 'properties': {'population': {'description': 'Total number of participants.',
   'title': 'Population',
   'type': 'integer'},
  'time_frame': {'description': 'Time frame',
   'title': 'Time Frame',
   'type': 'string'},
  'outcomes': {'items': {'$ref': '#/$defs/Outcome'},
   'title': 'Outcomes',
   'type': 'array'}},
 'required': ['population', 'time_frame', 'outcomes'],
 'title': 'ClinicalResult',
 'type': 'object'}

In [488]:
- **Population**: Evaluable patients (N=30)
- **Time Frame**: Dose expansion phase - Change from baseline up to 12 months.
- **Results**:
  * Stable Disease: 7 participants
  * Progressive Disease: 22 participants
  * Early Death From Malignant Disease: 1 participa

SyntaxError: invalid syntax (1536186896.py, line 1)

In [524]:
at_once = False
openai_client = OpenAI(
    base_url=os.getenv("BASE_URL"),
    api_key=os.getenv("OPENAI_API_KEY"),
    http_client=httpx.Client(verify=False)
)

chosen = all_studies[(all_studies.screen_eval>=0)&(all_studies.hasResults==True)]
chosen['res_with_part'] = chosen['outcomeMeasures'].apply(lambda x: [obj for obj in x \
                                                          if(obj.get('unitOfMeasure','').lower() in ['percentage of participants','participants'])])

to_work = chosen[chosen.res_with_part.str.len()>0]
if to_work.shape[0]:
    messages = []
    for i in to_work.res_with_part.values:
        n_outcomes = len(i)
        messages.append([{'role':'system', 'content':PROMPT_RES_EXTRACTION+' \no_think'},
                {'role':'user', 'content':f"{i}"}])
    if at_once:
        results = batch_call_openai(messages, os.getenv('MODEL_NAME'), 
                                    int(os.getenv('TEMPERATURE')), thinking=False)
    else:
        resultsct = []
        for i in messages:
            #$answer = extract.use_llm(os.getenv('MODEL_NAME'),i)
            
            response = openai_client.chat.completions.create(
                model=os.getenv('MODEL_NAME'),
                messages=messages[0],
                temperature=0,
                response_format={
                    "type": "json_object",
                    "json_schema": {
                        "name": "schema",
                        "schema": ClinicalResult.model_json_schema()
                    },
                },
            )
            fin = response.choices[0].message.content
            answer = fin.strip('<think>\n\n</think>\n\n')
            resultsct.append(answer)
    for i in resultsct:
        print(i)
        print('-----')

C:\Users\user\AppData\Local\Temp\ipykernel_4268\2758598942.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chosen['res_with_part'] = chosen['outcomeMeasures'].apply(lambda x: [obj for obj in x \


```json
[
    {
        "population": 30,
        "time_frame": "Dose Expansion phase: change from baseline and up to 12 months.",
        "outcomes": [
            {
                "category_name": "Stable Disease",
                "outcome": 7
            },
            {
                "category_name": "Progressive Disease",
                "outcome": 22
            },
            {
                "category_name": "Early death from malignant disease",
                "outcome": 1
            }
        ]
    }
]
```
-----


In [526]:
from trialmind.api import parse_json_outputs
parse_json_outputs(resultsct)

[[{'population': 30,
   'time_frame': 'Dose Expansion phase: change from baseline and up to 12 months.',
   'outcomes': [{'category_name': 'Stable Disease', 'outcome': 7},
    {'category_name': 'Progressive Disease', 'outcome': 22},
    {'category_name': 'Early death from malignant disease', 'outcome': 1}]}]]

In [336]:
to_work.res_with_part.values[0][0]['title']

'Clinical Response to Binimetinib Combined With PF-02341066'

In [330]:
to_work.res_with_part.values[0]

[{'type': 'PRIMARY',
  'title': 'Clinical Response to Binimetinib Combined With PF-02341066',
  'description': 'To investigate response to treatment with RPII dose of Binimetinib with Crizotinib (PF-02341066), in patients with a) RASMT CRC or b) RASWT/cMET mut amplified CRC or c) RASWT/c-MET over-expressed CRC, as defined by stable, partially or completely responding disease, per Response Evaluation Criteria In Solid Tumors Criteria (RECIST v1.1) for target lesions and assessed by CT: Complete Response (CR), Disappearance of all target lesions; Partial Response (PR), \\>=30% decrease in the sum of the longest diameter of target lesions; Stable Disease (SD), Neither sufficient shrinkage to qualify for PR nor sufficient increase (\\>=20%) to qualify for Progressive Disease; Overall Response (OR) = CR + PR + SD',
  'populationDescription': 'Evaluable patients for the primary outcome are those patients who complete a response assessment after cycle 1 of treatment, or who progress early on 

In [556]:
to_work#.res_with_part#.values

,hasResults,nctId,briefTitle,overallStatus,briefSummary,conditions,studyType,phases,interventions,outcomeMeasures,screen_eval,screening,res_with_part
2,True,NCT02510001,MEK and MET Inhibition in Colorectal Cancer,COMPLETED,This trial is designed to try two new cancer d...,"[Solid Tumor, Colorectal Cancer]",INTERVENTIONAL,[PHASE1],"[pf-02341066, pd-0325901, binimetinib]","[{'type': 'PRIMARY', 'title': 'Maximal Tolerat...",1,"{'evaluations': ['YES', 'NO', 'UNCERTAIN', 'YE...","[{'type': 'PRIMARY', 'title': 'Clinical Respon..."


In [339]:
'30' in str(to_work.res_with_part.values[0])

True

In [318]:
resultsct

['For the PRIMARY outcome measure "Clinical Response to Binimetinib Combined With PF-02341066":\n\n- **Population**: Evaluable patients (N=30)\n- **Time Frame**: Dose expansion phase - Change from baseline up to 12 months.\n- **Results**:\n  * Stable Disease: 7 participants\n  * Progressive Disease: 22 participants\n  * Early Death From Malignant Disease: 1 participa']

In [ ]:
at_once = False
openai_client = OpenAI(
    base_url=os.getenv("BASE_URL"),
    api_key=os.getenv("OPENAI_API_KEY"),
    http_client=httpx.Client(verify=False)
)

chosen = all_studies[(all_studies.screen_eval>=0)&(all_studies.hasResults==True)]
chosen['res_with_part'] = chosen['outcomeMeasures'].apply(lambda x: [obj for obj in x \
                                                          if(obj.get('unitOfMeasure','').lower() in ['percentage of participants','participants'])])

to_work = chosen[chosen.res_with_part.str.len()>0]
if to_work.shape[0]:
    messages = []
    for i in to_work.res_with_part.values[:2]:
        n_outcomes = len(i)
        print(n_outcomes)
        messages.append([{'role':'system', 'content':f'You are a helpful medical assistant. Answer with a concise description of the efficiency statistics for EACH of {n_outcomes} provided outcome measures /no_think'},
                {'role':'user', 'content':f"{i}"}])
    if at_once:
        results = batch_call_openai(messages, os.getenv('MODEL_NAME'), 
                                    int(os.getenv('TEMPERATURE')), thinking=False)
    else:
        resultsct = []
        for i in messages:
            #$answer = extract.use_llm(os.getenv('MODEL_NAME'),i)
            
            response = openai_client.chat.completions.create(
                model=os.getenv('MODEL_NAME'),
                messages=messages[0],
                temperature=0
            )
            fin = response.choices[0].message.content
            answer = fin.strip('<think>\n\n</think>\n\n')
            resultsct.append(answer)
    for i in resultsct:
        print(i)
        print('-----')

### results in a given format?

In [ ]:

class Info(BaseModel):
    diagnose: str = Field(..., description="База клинических испытаний")
    treatments: str = Field(..., description="Высшие позиции рейтинга таргетных препаратов")

parser = PydanticOutputParser(pydantic_object=Info)

In [ ]:
prompt = ChatPromptTemplate.from_messages([
    (
        "system", 
        dedent(
            """
            You are a helpful assistant. 
            Your goal is to extract related information from the text provided by the user.
            Return the information in the following JSON-format.
            {fmt}
            """
        )
    ),
    (
        "human", 
        dedent(
            """
            Text: {text}"
            """
        )
    ),
])
chain = (
    prompt 
    | llm_reasoning 
    | parser
)


response = await chain.ainvoke({"text": text, "fmt": parser.get_format_instructions()})
print(response)

## 4. papers

In [29]:
fin_condition = 'Colorectal cancer'
treatements_eng =  ['Crizotinib','Lorlatinib',]
pubmed_api_key= '6892c4129cef143ff92d11533848d2e0d908'#os.getenv("PUBMED_API_KEY", '')

In [31]:
search_api = PubmedAPIWrapper()
# page_size is the max number of records to return!!!! not pages!
tmp_inputs = {
        "page_size": 20,
        "keyword_map": {'conditions':[fin_condition], 
                        'treatments':[treatements_eng[0]]
                       },
        "keywords": {
            "OPERATOR": 'AND'
        }
}

time.sleep(.5)
response = search_api.build_search_query_and_get_pmid(tmp_inputs, 
                                                      api_key=pubmed_api_key)
response

(['40526090',
  '38711893',
  '40140597',
  '35343863',
  '41213063',
  '36129915',
  '34898475',
  '36801912',
  '37418240',
  '36316649',
  '40369167',
  '36053834',
  '37686423',
  '40211189',
  '38212428',
  '36627781',
  '37773318',
  '38378070',
  '41268440',
  '37543570'],
 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&term=(Colorectal cancer)+AND+(Crizotinib)&retmax=20&retmode=json&api_key=6892c4129cef143ff92d11533848d2e0d908',
 '67')

In [32]:
df_papers = pmid2papers(#pmid_list=['40526090', '41213063', '41268440'],
                        pmid_list=response[0], 
                        api_key=pubmed_api_key)
df_papers[0]

,PMID,Journal,Year,Month,Day,Title,Publication Type,Authors,Abstract
0,40526090,Oncotarget,2025,Jun,17,Case Report WIN-MTB-2023001 WIN International ...,"Journal Article, Case Reports","Alberto Hernando-Calvo, Razelle Kurzrock, Nadi...",Heavily pretreated metastatic colorectal cance...
1,38711893,AME case reports,2024,,,Partial response to crizotinib + regorafenib +...,"Case Reports, Journal Article","Yingying Huang, Shuai Zhang, Xueqing Hu, Xiang...",Colorectal cancer (CRC) with the Raf murine sa...
2,40140597,BJC reports,2025,Mar,26,A phase Ia study of the MEK1/2 inhibitor PD-03...,Journal Article,"Peter Gallagher, Christian Rolfo, Elena Elez, ...",Single-agent MEK1/2 inhibition has been disapp...
3,35343863,Expert opinion on therapeutic patents,2022,Jun,,An updated patent review of small-molecule ROS...,"Journal Article, Review","Meng Liu, Jintian Dai, Mudan Wei, Qingshan Pan...",C-ros oncogene 1 (ROS1) is the sole member of ...
4,41213063,"Journal of environmental pathology, toxicology...",2025,,,Radiotherapy-Related Autophagy Genes Predict P...,Journal Article,"Mao Mao, Yichao Zhuang, Yan Chen",Numerous genes have been associated with color...
5,36129915,PloS one,2022,,,Investigation of cell signalings and therapeut...,"Journal Article, Research Support, Non-U.S. Gov't","Jae Heon Jeong, Jae Won Yun, Ha Young Kim, Cha...",Colorectal cancer (CRC) is one of the most dea...
6,34898475,Aging,2021,Dec,13,Identification of a novel immune signature for...,"Journal Article, Research Support, Non-U.S. Gov't","Yan Li, Yiyi Li, Zijin Xia, Dun Zhang, Xiaomei...","Globally, colorectal cancer (CRC) is one of th..."
7,36801912,NPJ precision oncology,2023,Feb,18,BRAF v600E-mutant cancers treated with vemuraf...,Journal Article,"Blessie Elizabeth Nelson, Jason Roszik, Filip ...",Combined BRAF + MEK inhibition is FDA approved...
8,37418240,ACS applied materials & interfaces,2023,Jul,19,Albumin-Based Cyanine Crizotinib Conjugate Nan...,Journal Article,"Zhuang Hu, Ruihan Li, Xinyue Cui, Zilin Chen",Colorectal cancer (CRC) is presently the third...
9,36316649,BMC cancer,2022,Oct,31,Personalized targeted therapy prescription in ...,Journal Article,"Maxim Sorokin, Marianna Zolotovskaia, Daniil N...",Overall survival of advanced colorectal cancer...


### papers screen criterias

In [33]:
# test the apis

api = ScreeningCriteriaGeneration()
ecs_p = api.run(
    population = f"Patients with {fin_condition} undergoing treatment with {treatements_eng[0]}",
    intervention = f"{treatements_eng[0]}",
    comparator = "",
    outcome = "",
    llm = os.getenv("MODEL_NAME"),
    num_title_criteria=2,
    num_abstract_criteria=2,
    thinking=False,
)
ecs_p

call_openai https://gigachat.devices.sberbank.ru/api/v1
api_call_single https://gigachat.devices.sberbank.ru/api/v1/
```json
{
    "ELIGIBILITY_ANALYSIS": [
        "Patients must have colorectal cancer to ensure relevance to the study population",
        "Crizotinib must be used as an intervention to focus on the specific drug being studied",
        "Comparisons must be present to allow for meaningful contrast between treatments",
        "Outcomes must be clearly defined to enable consistent evaluation across studies",
        "Studies must be randomized controlled trials or observational studies to maintain high-quality evidence"
    ],
    "TITLE_CRITERIA": [
        "Does the title mention 'colorectal cancer'?",
        "Does the title mention 'Crizotinib'?"
    ],
    "CONTENT_CRITERIA": [
        "Is there a clear description of patient characteristics including diagnosis and stage of disease?",
        "Are the methods section and results section sufficiently detailed to asse

{'title_criteria': ["Does the title mention 'colorectal cancer'?",
  "Does the title mention 'Crizotinib'?"],
 'content_criteria': ['Is there a clear description of patient characteristics including diagnosis and stage of disease?',
  'Are the methods section and results section sufficiently detailed to assess the quality and validity of the study?'],
 'eligibility_analysis': ['Patients must have colorectal cancer to ensure relevance to the study population',
  'Crizotinib must be used as an intervention to focus on the specific drug being studied',
  'Comparisons must be present to allow for meaningful contrast between treatments',
  'Outcomes must be clearly defined to enable consistent evaluation across studies',
  'Studies must be randomized controlled trials or observational studies to maintain high-quality evidence']}

In [34]:
papers = df_papers[0]["Title"] + ": " + df_papers[0]["Abstract"].fillna("") # important to fillna
papers = papers.tolist()
papers[:1]

['Case Report WIN-MTB-2023001 WIN International Molecular Tumor Board A 62-year-old male with metastatic colorectal cancer with 5 prior lines of treatment.: Heavily pretreated metastatic colorectal cancer (mCRC) poses significant therapeutic challenges. Advances in molecular profiling enables personalized strategies. We present a 62-year-old male with mCRC harboring']

In [35]:
'''
'title_criteria': ["Does the title mention 'colorectal cancer'?",
  "Does the title mention 'Crizotinib'?"],
'content_criteria': ['Is there evidence of Crizotinib administration in the study methods section?',
  'Are outcomes related to patient response, adverse events, or survival discussed in the results section?'],
'''
title_criteria = ecs_p['title_criteria']
content_criteria = ecs_p['content_criteria']
title_criteria+content_criteria

["Does the title mention 'colorectal cancer'?",
 "Does the title mention 'Crizotinib'?",
 'Is there a clear description of patient characteristics including diagnosis and stage of disease?',
 'Are the methods section and results section sufficiently detailed to assess the quality and validity of the study?']

### __check

In [15]:
from trialmind.llm import _batch_inputs_to_messages

In [22]:
from trialmind.prompts.screening import LITERATURE_SCREENING_FC
population = f"Patients with {fin_condition} undergoing treatment with {treatements_eng[0]}",
intervention = f"{treatements_eng[0]}",
comparator = "",
outcome = "",
criteria = title_criteria + content_criteria
criteria_text = [f"{idx+1}. {c}" for idx, c in enumerate(criteria)]
n_criteria = len(criteria_text)
    
batch_inputs = []
for paper in papers[:1]:
    batch_inputs.append({
        "P": population,
        "I": intervention,
        "C": comparator,
        "O": outcome,
        "paper_content": paper,
        "criteria_text": criteria_text,
        "num_criteria": n_criteria
    })
            
LITERATURE_SCREENING_FC

'\n# CONTEXT #\nYou are a clinical specialist tasked with assessing research papers for inclusion in a meta-analysis based on specific eligibility criteria.\n\n# OBJECTIVE #\nEvaluate each criterion of a given paper to determine its eligibility for inclusion in the meta-analysis. Provide a list of decisions ("YES", "NO", or "UNCERTAIN") for each eligibility criterion. You must deliver exactly {num_criteria} responses.\n\n# IMPORTANT NOTE #\nIf the information within the provided paper content is insufficient to conclusively evaluate a criterion, you must opt for "UNCERTAIN" as your response. Avoid making assumptions or extrapolating beyond the provided data, as accurate and reliable responses are crucial, and fabricating information (hallucinations) could lead to serious errors in the meta-analysis.\n\n# PICO FRAMEWORK #\n- P (Patient, Problem or Population): {P}\n- I (Intervention): {I}\n- C (Comparison): {C}\n- O (Outcome): {O}\n\n# PAPER DETAILS #\n- Provided Paper: {paper_content}\

In [37]:
tools

[{'type': 'function',
  'function': {'name': 'PaperEvaluation',
   'description': '',
   'parameters': {'properties': {'evaluations': {'description': 'Evaluations for 4 criteria, must be of length 4',
      'items': {'enum': ['YES', 'NO', 'UNCERTAIN'], 'type': 'string'},
      'maxItems': 4,
      'minItems': 4,
      'type': 'array'},
     'rationale': {'description': 'A rationale for each criteria evaluation',
      'items': {'type': 'string'},
      'maxItems': 4,
      'minItems': 4,
      'type': 'array'}},
    'required': ['evaluations', 'rationale'],
    'type': 'object'}}}]

In [45]:
messages = _batch_inputs_to_messages(prompt_template=LITERATURE_SCREENING_FC, batch_inputs=batch_inputs)

from openai import AsyncOpenAI
tools = [{'type': 'function', 'function': {'name': 'PaperEvaluation', 'description': '', 'parameters': {'properties': {'evaluations': {'description': 'Evaluations for 4 criteria, must be of length 4', 'items': {'enum': ['YES', 'NO', 'UNCERTAIN'], 'type': 'string'}, 'maxItems': 4, 'minItems': 4, 'type': 'array'}, 'rationale': {'description': 'A rationale for each criteria evaluation', 'items': {'type': 'string'}, 'maxItems': 4, 'minItems': 4, 'type': 'array'}}, 'required': ['evaluations', 'rationale'], 'type': 'object'}}}]

async_openai_client = AsyncOpenAI(
        base_url=os.getenv("BASE_URL"),
        api_key=os.getenv("OPENAI_API_KEY"),
        http_client=httpx.AsyncClient(verify=False)
    )
response = await async_openai_client.chat.completions.create(
    model=os.getenv("MODEL_NAME"),
    messages=messages[0],
    tools=tools,
    temperature=0
)
response.choices[0].message

ChatCompletionMessage(content='```json\n{\n  "evaluations": [\n    "YES",\n    "YES",\n    "UNCERTAIN",\n    "YES",\n    "YES",\n    "UNCERTAIN"\n  ],\n  "rationale": [\n    "The title mentions \'metastatic colorectal cancer\'",\n    "The title mentions \'Crizotinib\'",\n    "There is no comparison mentioned in the title",\n    "Methods and results sections are briefly described but not detailed enough to confirm clarity",\n    "The text indicates adenocarcinoma as the type of colorectal cancer",\n    "No explicit statement about outcome duration is made"\n  ]\n}\n```  \n\n### Explanation of Decisions:\n1. **Does the title mention \'colorectal cancer\'?**  \n   *YES* — The title states “metastatic colorectal cancer.”\n   \n2. **Does the title mention \'crizotinib\'?**  \n   *YES* — The term “Crizotinib” appears in the title.\n\n3. **Is there any indication in the title that the study compares crizotinib to another treatment?**  \n   *UNCERTAIN* — There’s no direct comparison indicated 

In [52]:
response.choices[0].message.content

'```json\n{\n  "evaluations": [\n    "YES",\n    "YES",\n    "UNCERTAIN",\n    "YES",\n    "YES",\n    "UNCERTAIN"\n  ],\n  "rationale": [\n    "The title mentions \'metastatic colorectal cancer\'",\n    "The title mentions \'Crizotinib\'",\n    "There is no comparison mentioned in the title",\n    "Methods and results sections are briefly described but not detailed enough to confirm clarity",\n    "The text indicates adenocarcinoma as the type of colorectal cancer",\n    "No explicit statement about outcome duration is made"\n  ]\n}\n```  \n\n### Explanation of Decisions:\n1. **Does the title mention \'colorectal cancer\'?**  \n   *YES* — The title states “metastatic colorectal cancer.”\n   \n2. **Does the title mention \'crizotinib\'?**  \n   *YES* — The term “Crizotinib” appears in the title.\n\n3. **Is there any indication in the title that the study compares crizotinib to another treatment?**  \n   *UNCERTAIN* — There’s no direct comparison indicated in the title.\n\n4. **Are the 

In [60]:
from trialmind.api import extract_json,parse_json_outputs

In [62]:
parse_json_outputs([response.choices[0].message.content])

[{'evaluations': ['YES', 'YES', 'UNCERTAIN', 'YES', 'YES', 'UNCERTAIN'],
  'rationale': ["The title mentions 'metastatic colorectal cancer'",
   "The title mentions 'Crizotinib'",
   'There is no comparison mentioned in the title',
   'Methods and results sections are briefly described but not detailed enough to confirm clarity',
   'The text indicates adenocarcinoma as the type of colorectal cancer',
   'No explicit statement about outcome duration is made']}]

### 5. paper screen


In [109]:

api = LiteratureScreening()
ec_predP = api.run(
    population = f"Patients with {fin_condition} undergoing treatment with {treatements_eng[0]}",
    intervention = f"{treatements_eng[0]}",
    comparator = "",
    outcome = "",
    llm = os.getenv("MODEL_NAME"),
    criteria = ["Does the content mention 'colorectal cancer'?",
                 "Does content mention 'Crizotinib'?", 
                'Is there a description of treatement results?'],#title_criteria + content_criteria,
    papers = papers, # make for the top-100 for demo
)
ec_predP

1

parsed_results in asynch ['```json\n{\n    "evaluations": ["YES", "YES", "UNCERTAIN"],\n    "rationale": [\n        "The paper mentions \'metastatic colorectal cancer\', which implies colorectal cancer.",\n        "The term \'Crizotinib\' is explicitly mentioned in the title and abstract.",\n        "There is no detailed description of treatment outcomes; only brief mention of treatment."\n    ]\n}']

parsed_results in asynch ['```json\n{\n    "evaluations": ["YES", "YES", "YES"],\n    "rationale": [\n        "The paper mentions \'ascending colon adenocarcinoma\', which is part of colorectal cancer.",\n        "The term \'Crizotinib\' is explicitly mentioned in the title and throughout the abstract and body of the paper.",\n        "The paper describes treatment outcomes including partial response to Crizotinib, regorafenib, and PD-1 inhibitor."\n    ]\n}\n```']

parsed_results in asynch ['```json\n{\n    "evaluations": ["YES", "YES", "YES"],\n    "rationale": [\n        "The paper 

[{'evaluations': ['YES', 'YES', 'UNCERTAIN'],
  'rationale': ["The paper mentions 'metastatic colorectal cancer', which implies colorectal cancer.",
   "The term 'Crizotinib' is explicitly mentioned in the title and abstract.",
   'There is no detailed description of treatment outcomes; only brief mention of treatment.']},
 {'evaluations': ['YES', 'YES', 'YES'],
  'rationale': ["The paper mentions 'ascending colon adenocarcinoma', which is part of colorectal cancer.",
   "The term 'Crizotinib' is explicitly mentioned in the title and throughout the abstract and body of the paper.",
   'The paper describes treatment outcomes including partial response to Crizotinib, regorafenib, and PD-1 inhibitor.']},
 {'evaluations': ['YES', 'YES', 'YES'],
  'rationale': ["The paper mentions 'advanced solid cancers', but explicitly states 'Patients with Colorectal cancer undergoing treatment with Crizotinib'",
   "The intervention described is 'Crizotinib'",
   "The paper reports 'best clinical respon

In [110]:
evalsP = [i['evaluations'] for i in ec_predP]

word2int = {"YES": 1, 
            "UNCERTAIN": 0,
            "NO": -1
           }
#rev_subs = { v:k for k, v in word2int.items()} # subs.iteritems() In Python 3
new_evalsP = []
for one_e in evalsP:
    new_evalsP.append([word2int.get(item, 0) for item in one_e ])
new_evalsP = np.array(new_evalsP)    
print(new_evalsP)
print(new_evalsP.sum(axis=1))
df_papers[0]['screen_eval'] = new_evalsP.sum(axis=1)
df_papers[0]

[[ 1  1  0]
 [ 1  1  1]
 [ 1  1  1]
 [ 1  1  0]
 [ 1  1  1]
 [ 1  1  0]
 [ 1  1  1]
 [-1  1  0]
 [ 1  1  1]
 [ 1  1  1]
 [ 1  1  1]
 [ 1  1  1]
 [ 1  1  0]
 [ 1  1  1]
 [ 1  1  1]
 [ 1  1  0]
 [ 1  1  1]
 [ 1  1  1]
 [-1  1  0]
 [ 1  1  1]]
[2 3 3 2 3 2 3 0 3 3 3 3 2 3 3 2 3 3 0 3]


,PMID,Journal,Year,Month,Day,Title,Publication Type,Authors,Abstract,screen_eval,FullText,screening,res_extr
0,40526090,Oncotarget,2025,Jun,17,Case Report WIN-MTB-2023001 WIN International ...,"Journal Article, Case Reports","Alberto Hernando-Calvo, Razelle Kurzrock, Nadi...",Heavily pretreated metastatic colorectal cance...,2,Case Report WIN-MTB-2023001 WIN International ...,"{'evaluations': ['YES', 'YES', 'YES', 'UNCERTA...",
1,38711893,AME case reports,2024,,,Partial response to crizotinib + regorafenib +...,"Case Reports, Journal Article","Yingying Huang, Shuai Zhang, Xueqing Hu, Xiang...",Colorectal cancer (CRC) with the Raf murine sa...,3,Partial response to crizotinib + regorafenib +...,"{'evaluations': ['YES', 'YES', 'YES', 'UNCERTA...",
2,40140597,BJC reports,2025,Mar,26,A phase Ia study of the MEK1/2 inhibitor PD-03...,Journal Article,"Peter Gallagher, Christian Rolfo, Elena Elez, ...",Single-agent MEK1/2 inhibition has been disapp...,3,A phase Ia study of the MEK1/2 inhibitor PD-03...,"{'evaluations': ['YES', 'YES', 'YES', 'YES'], ...",
3,35343863,Expert opinion on therapeutic patents,2022,Jun,,An updated patent review of small-molecule ROS...,"Journal Article, Review","Meng Liu, Jintian Dai, Mudan Wei, Qingshan Pan...",C-ros oncogene 1 (ROS1) is the sole member of ...,2,An updated patent review of small-molecule ROS...,"{'evaluations': ['YES', 'YES', 'UNCERTAIN', 'U...",
4,41213063,"Journal of environmental pathology, toxicology...",2025,,,Radiotherapy-Related Autophagy Genes Predict P...,Journal Article,"Mao Mao, Yichao Zhuang, Yan Chen",Numerous genes have been associated with color...,3,Radiotherapy-Related Autophagy Genes Predict P...,"{'evaluations': ['YES', 'YES', 'YES', 'YES'], ...",
5,36129915,PloS one,2022,,,Investigation of cell signalings and therapeut...,"Journal Article, Research Support, Non-U.S. Gov't","Jae Heon Jeong, Jae Won Yun, Ha Young Kim, Cha...",Colorectal cancer (CRC) is one of the most dea...,2,Investigation of cell signalings and therapeut...,"{'evaluations': ['YES', 'YES', 'UNCERTAIN', 'Y...",
6,34898475,Aging,2021,Dec,13,Identification of a novel immune signature for...,"Journal Article, Research Support, Non-U.S. Gov't","Yan Li, Yiyi Li, Zijin Xia, Dun Zhang, Xiaomei...","Globally, colorectal cancer (CRC) is one of th...",3,Identification of a novel immune signature for...,"{'evaluations': ['YES', 'YES', 'YES', 'YES'], ...",
7,36801912,NPJ precision oncology,2023,Feb,18,BRAF v600E-mutant cancers treated with vemuraf...,Journal Article,"Blessie Elizabeth Nelson, Jason Roszik, Filip ...",Combined BRAF + MEK inhibition is FDA approved...,0,,"{'evaluations': ['NO', 'YES', 'UNCERTAIN', 'NO...",
8,37418240,ACS applied materials & interfaces,2023,Jul,19,Albumin-Based Cyanine Crizotinib Conjugate Nan...,Journal Article,"Zhuang Hu, Ruihan Li, Xinyue Cui, Zilin Chen",Colorectal cancer (CRC) is presently the third...,3,Albumin-Based Cyanine Crizotinib Conjugate Nan...,"{'evaluations': ['YES', 'YES', 'UNCERTAIN', 'Y...",
9,36316649,BMC cancer,2022,Oct,31,Personalized targeted therapy prescription in ...,Journal Article,"Maxim Sorokin, Marianna Zolotovskaia, Daniil N...",Overall survival of advanced colorectal cancer...,3,Personalized targeted therapy prescription in ...,"{'evaluations': ['YES', 'YES', 'YES', 'YES'], ...",


### 6. papers results extraction
Только для статей с оценкой выше 0 !! Чтобы не загружать полный текст у всех

+ надо посмотреть: если фул текст недоступен, то загружается ли абстракт

In [113]:
len(pmid_list)

18

In [244]:
df_papers[0][df_papers[0].screen_eval>=3].shape

(13, 13)

In [262]:
total_ev = 3
pmid_list = df_papers[0][df_papers[0].screen_eval>=total_ev].PMID.values.tolist()
#['41213063',#'26451310',]

res = pmid2biocxml(pmid_list, api_key=pubmed_api_key)
#print(res)
res = [parse_bioc_xml(r) for r in res]
# transform the parsed xml into paper content
papers0 = []
for parsed in res:
    paper_content = []
    for parsed_ in parsed["passage"]:
        paper_content.append(parsed_['content'])
    paper_content = "\n".join(paper_content)
    papers0.append(paper_content)

df_papers[0]['FullText'] = ''
df_papers[0].loc[df_papers[0].screen_eval>=total_ev, 'FullText'] = papers0


In [455]:
df_papers[0][df_papers[0].screen_eval>=total_ev]

,PMID,Journal,Year,Month,Day,Title,Publication Type,Authors,Abstract,screen_eval,FullText,screening,res_extr
1,38711893,AME case reports,2024,,,Partial response to crizotinib + regorafenib +...,"Case Reports, Journal Article","Yingying Huang, Shuai Zhang, Xueqing Hu, Xiang...",Colorectal cancer (CRC) with the Raf murine sa...,3,Partial response to crizotinib + regorafenib +...,"{'evaluations': ['YES', 'YES', 'YES'], 'ration...",
2,40140597,BJC reports,2025,Mar,26,A phase Ia study of the MEK1/2 inhibitor PD-03...,Journal Article,"Peter Gallagher, Christian Rolfo, Elena Elez, ...",Single-agent MEK1/2 inhibition has been disapp...,3,A phase Ia study of the MEK1/2 inhibitor PD-03...,"{'evaluations': ['YES', 'YES', 'YES'], 'ration...",
4,41213063,"Journal of environmental pathology, toxicology...",2025,,,Radiotherapy-Related Autophagy Genes Predict P...,Journal Article,"Mao Mao, Yichao Zhuang, Yan Chen",Numerous genes have been associated with color...,3,Radiotherapy-Related Autophagy Genes Predict P...,"{'evaluations': ['YES', 'YES', 'YES'], 'ration...",
6,34898475,Aging,2021,Dec,13,Identification of a novel immune signature for...,"Journal Article, Research Support, Non-U.S. Gov't","Yan Li, Yiyi Li, Zijin Xia, Dun Zhang, Xiaomei...","Globally, colorectal cancer (CRC) is one of th...",3,Identification of a novel immune signature for...,"{'evaluations': ['YES', 'YES', 'YES'], 'ration...",
8,37418240,ACS applied materials & interfaces,2023,Jul,19,Albumin-Based Cyanine Crizotinib Conjugate Nan...,Journal Article,"Zhuang Hu, Ruihan Li, Xinyue Cui, Zilin Chen",Colorectal cancer (CRC) is presently the third...,3,Albumin-Based Cyanine Crizotinib Conjugate Nan...,"{'evaluations': ['YES', 'YES', 'YES'], 'ration...",
9,36316649,BMC cancer,2022,Oct,31,Personalized targeted therapy prescription in ...,Journal Article,"Maxim Sorokin, Marianna Zolotovskaia, Daniil N...",Overall survival of advanced colorectal cancer...,3,Personalized targeted therapy prescription in ...,"{'evaluations': ['YES', 'YES', 'YES'], 'ration...",
10,40369167,Cell death and differentiation,2025,Nov,,ARID1A loss enhances sensitivity to c-MET inhi...,Journal Article,"Xu Zhang, Zihuan Wang, Yilin He, Kejin Wang, C...","ARID1A, a subunit of the SWI/SNF chromatin-rem...",3,ARID1A loss enhances sensitivity to c-MET inhi...,"{'evaluations': ['YES', 'YES', 'YES'], 'ration...",
11,36053834,International journal of cancer,2022,Dec,15,ROS1 genomic rearrangements are rare actionabl...,Journal Article,"Dilara Akhoundova, Saskia Hussung, Smruthy Siv...","c-Ros oncogene 1, receptor tyrosine kinase (RO...",3,ROS1 genomic rearrangements are rare actionabl...,"{'evaluations': ['YES', 'YES', 'YES'], 'ration...",
13,40211189,BMC cancer,2025,Apr,10,A Phase Ia/b study of MEK1/2 inhibitor binimet...,"Journal Article, Clinical Trial, Phase I","Francesca Aroldi, Elena Elez, Thierry André, G...",Targeting RAS mutant (MT) colorectal cancer (C...,3,A Phase Ia/b study of MEK1/2 inhibitor binimet...,"{'evaluations': ['YES', 'YES', 'YES'], 'ration...",
14,38212428,Cellular and molecular life sciences : CMLS,2024,Jan,12,Inhibition of autocrine HGF maturation overcom...,Journal Article,"Vivian Truong Jones, Ramona Graves-Deal, Zheng...",Although amplifications and mutations in recep...,3,Inhibition of autocrine HGF maturation overcom...,"{'evaluations': ['YES', 'YES', 'YES'], 'ration...",


In [448]:
pmid_list

['38711893',
 '40140597',
 '41213063',
 '34898475',
 '37418240',
 '36316649',
 '40369167',
 '36053834',
 '40211189',
 '38212428',
 '37773318',
 '38378070',
 '37543570']

In [447]:
res[0]

{'passage': [{'pmid': '',
   'pmc': '',
   'section': '',
   'content': 'Partial response to crizotinib + regorafenib + PD-1 inhibitor in a metastatic BRAF V600EMT colon cancer patient with acquired C-MET amplification and TPM4-ALK fusion: a case report.'},
  {'pmid': '',
   'pmc': '',
   'section': '',
   'content': "Background: Colorectal cancer (CRC) with the Raf murine sarcoma viral oncogene homolog B (BRAF) V600E had a relatively poor prognosis. Anaplastic lymphoma kinase (ALK) fusion and the mesenchymal-to-epithelial transition factor (MET) amplification have been recognized as potentially important therapeutic targets in non-small cell lung cancer (NSCLC). However, both of them are of extremely lower frequencies (<2%) in metastatic CRC, and few studies have mentioned the real application of their inhibitors in CRC treatment. Case Description: A 49-year-old Chinese male was diagnosed with ascending colon adenocarcinoma (cT3N+?M1) with liver metastases. The patient performed next-

In [277]:
df_papers[0].FullText.str.len()#.sum()

0        0
1     2665
2     1612
3        0
4     1652
5        0
6     1792
7        0
8     1637
9     1951
10    1895
11    1817
12       0
13    2627
14    1402
15       0
16    1606
17    2064
18       0
19    2078
Name: FullText, dtype: int64

In [442]:
df_papers[0][df_papers[0].screen_eval>=total_ev].loc[8]

PMID                                                         37418240
Journal                            ACS applied materials & interfaces
Year                                                             2023
Month                                                             Jul
Day                                                                19
Title               Albumin-Based Cyanine Crizotinib Conjugate Nan...
Publication Type                                      Journal Article
Authors                  Zhuang Hu, Ruihan Li, Xinyue Cui, Zilin Chen
Abstract            Colorectal cancer (CRC) is presently the third...
screen_eval                                                         3
FullText            Albumin-Based Cyanine Crizotinib Conjugate Nan...
screening           {'evaluations': ['YES', 'YES', 'YES'], 'ration...
res_extr                                                             
Name: 8, dtype: object

In [441]:
papers0[4]

'Albumin-Based Cyanine Crizotinib Conjugate Nanoparticles for NIR-II Imaging-Guided Synergistic Chemophototherapy.\nColorectal cancer (CRC) is presently the third deadliest cancer in the world. This malignant cancer usually precedes the progression of precancerous lesions, and it is challenging to distinguish its nuanced morphological changes. Molecular-based near-infrared-II (NIR-II) fluorescence imaging can effectively recognize lesion targets to improve image contrast and increase early tumor detection compared with traditional wide-light screening endoscopy. c-Met has been determined to be overexpressed in advanced stages of CRC and is considered to be a potent tumor biomarker. Herein, based on the well-targeted inhibitory effect of Crizotinib on c-Met positive tumor cells, the dye IR808 was covalently combined with the drug molecule Crizotinib, resulting in the synthesis of a NIR fluorescent probe Crizotinib-IR808 targeting c-Met positive tumor cells. Then, water-insoluble Crizoti

In [443]:
splited = split_text_into_chunks(papers0[4], chunk_size=500,
                                             chunk_overlap=200)
splited

['Albumin-Based Cyanine Crizotinib Conjugate Nanoparticles for NIR-II Imaging-Guided Synergistic Chemophototherapy.',
 'Colorectal cancer (CRC) is presently the third deadliest cancer in the world. This malignant cancer usually precedes the progression of precancerous lesions, and it is challenging to distinguish its nuanced morphological changes. Molecular-based near-infrared-II (NIR-II) fluorescence imaging can effectively recognize lesion targets to improve image contrast and increase early tumor detection compared with traditional wide-light screening endoscopy. c-Met has been determined to be overexpressed',
 'recognize lesion targets to improve image contrast and increase early tumor detection compared with traditional wide-light screening endoscopy. c-Met has been determined to be overexpressed in advanced stages of CRC and is considered to be a potent tumor biomarker. Herein, based on the well-targeted inhibitory effect of Crizotinib on c-Met positive tumor cells, the dye IR808

In [87]:
len(papers0)

18

### __check

In [ ]:
{
     "fieldresult": [
       {
         "name": "Title",
         "value": "Основы программирования на Python",
         "source_id": [104]
       },
       {
         "name": "Pages",
         "value": "300 страниц",
         "source_id": [105]
       }
     ]
   }

In [ ]:

        Format:
        ```json
        [
            {{
                "name":  \\ str, length <= 25 tokens
                "value":  \\ str, length <= 25 tokens
                "source_id":  \\ list[int], length <= 3 ids
            }},
            {{
                "name":, \\ str, length <= 25 tokens
                "value":  \\ str, length <= 25 tokens
                "source_id":  \\ list[int], length <= 3 ids
            }},
            ...
        ]
        ```
    \"\"\"
```
    
    
new
```json
        [
            {{
                "name": "fieldName1",
                "value": "extractedInfo1",
                "source_id":[source1, source2, ...]
            }},
            {{
                "name":["fieldName2"],
                "value":["extractedInfo2"],
                "source_id":[source1, source2, ...]
            }},
            ...
        ]
```

In [26]:
from pydantic import BaseModel, validator, Field, conlist  # This is the new version
from typing import Dict, Literal
class FieldResult(BaseModel):
    name: str = Field(description='Field name that accurately represents the content of the field based on its description.',
                     max_length=25)
    value: str = Field(description='Extracted information from the text based on the field description.',
                      max_length=50)
    source_id: conlist(int,min_length=1, max_length=3) = Field(description='Cited document IDs.')
    
class Results(BaseModel):
    fieldresult: FieldResult
    
Results.model_json_schema()

{'$defs': {'FieldResult': {'properties': {'name': {'description': 'Field name that accurately represents the content of the field based on its description.',
     'maxLength': 25,
     'title': 'Name',
     'type': 'string'},
    'value': {'description': 'Extracted information from the text based on the field description.',
     'maxLength': 50,
     'title': 'Value',
     'type': 'string'},
    'source_id': {'description': 'Cited document IDs.',
     'items': {'type': 'integer'},
     'maxItems': 3,
     'minItems': 1,
     'title': 'Source Id',
     'type': 'array'}},
   'required': ['name', 'value', 'source_id'],
   'title': 'FieldResult',
   'type': 'object'}},
 'properties': {'fieldresult': {'$ref': '#/$defs/FieldResult'}},
 'required': ['fieldresult'],
 'title': 'Results',
 'type': 'object'}

In [69]:
from pydantic import BaseModel, validator, Field, conlist,   # This is the new version
from typing import Dict, Literal
import pydantic
import random
class Nested(pydantic.BaseModel):
    a: str = pydantic.Field(examples=["hello"])


class ExampleData(pydantic.BaseModel):
    a: int = pydantic.Field(examples=[1])
    b: str = pydantic.Field(min_length=10, max_length=10, examples=["abc-123-45"])
    c: int = pydantic.Field(default=100)
    # Note: Example will be populated by child, so no explicit example needs
    # to be provided for nested models
    n: Nested
    
nn = 1    
class FieldResult(BaseModel):
    name: str = Field(description='Field name that accurately represents the content of the field based on its description.',
                     max_length=25, default='effectiveness')
    value: str = Field(description='Extracted information from the text based on the field description.',
                      max_length=50, default='total of 5% died')
    #source_id: conlist(int,min_length=1, max_length=3) = Field(description='Cited document IDs.', examples=[1,3])
    
class Results(BaseModel):
    fieldresult: list[FieldResult]

    
def populate_with_examples_recursive(model: pydantic.BaseModel) -> dict:
    """Populate fields with example values"""

    result = {}
    
    for name, field_info in model.model_fields.items():
        if issubclass(field_info.annotation, pydantic.BaseModel):
            result[name] = populate_with_examples_recursive(field_info.annotation)
        elif field_info.examples:
            result[name] = random.choice(field_info.examples)
        elif field_info.default:
            result[name] = field_info.default

    return dict(result)


# Execute it:
#json.loads(f'''{populate_with_examples_recursive(ExampleData)}''')

(populate_with_examples_recursive(Results))

SyntaxError: trailing comma not allowed without surrounding parentheses (1178800490.py, line 1)

In [83]:
class FieldResult(BaseModel):
    name: str = Field(description='Field name that accurately represents the content of the field based on its description.',
                     max_length=25, default='effectiveness')
    value: str = Field(description='Extracted information from the text based on the field description.',
                      max_length=50, default='total of 5% died')
    source_id: list[int] = Field(description='Cited document IDs.', examples=[1,3])
    #conlist ,min_length=1, max_length=3
class Results(BaseModel):
    fieldresult: list[FieldResult]

a1 = FieldResult()
a = Results(fieldresult=[a1,a1])

print(a.model_dump_json(indent=4))

ValidationError: 1 validation error for FieldResult
source_id
  Field required [type=missing, input_value={}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.12/v/missing

In [27]:
papers[:2]

['Partial response to crizotinib + regorafenib + PD-1 inhibitor in a metastatic: Colorectal cancer (CRC) with the Raf murine sarcoma viral oncogene homolog B (\nA 49-year-old Chinese male was diagnosed with ascending colon adenocarcinoma (cT3N+?M1) with liver metastases. The patient performed next-generation sequencing (NGS) using tissue and circulating tumor DNA (ctDNA), and the results showed a\nThis study reported a co-existence of a',
 'ARID1A loss enhances sensitivity to c-MET inhibition by dual targeting of GPX4 and iron homeostasis, inducing ferroptosis.: ARID1A, a subunit of the SWI/SNF chromatin-remodeling complex, functions as a tumor suppressor in various cancer types. Owing to its high frequency of inactivating mutations, ARID1A has emerged as a promising target for the development of anticancer drugs. In this study, we report that ARID1A-deficient colorectal cancer (CRC) cells induce synthetic lethality when treated with inhibitors of c-MET receptor tyrosine kinase. c-MET 

['',
 'Partial response to crizotinib + regorafenib + PD-1 inhibitor in a metastatic: Colorectal cancer (CRC) with the Raf murine sarcoma viral oncogene homolog B (\nA 49-year-old Chinese male was diagnosed with ascending colon adenocarcinoma (cT3N+?M1) with liver metastases. The patient performed next-generation sequencing (NGS) using tissue and circulating tumor DNA (ctDNA), and the results showed a\nThis study reported a co-existence of a',
 'ARID1A loss enhances sensitivity to c-MET inhibition by dual targeting of GPX4 and iron homeostasis, inducing ferroptosis.: ARID1A, a subunit of the SWI/SNF chromatin-remodeling complex, functions as a tumor suppressor in various cancer types. Owing to its high frequency of inactivating mutations, ARID1A has emerged as a promising target for the development of anticancer drugs. In this study, we report that ARID1A-deficient colorectal cancer (CRC) cells induce synthetic lethality when treated with inhibitors of c-MET receptor tyrosine kinase. c

### back

In [278]:
treatements_eng

['Crizotinib', 'Lorlatinib']

In [279]:
len(papers0)

13

In [445]:
papers0[0]

"Partial response to crizotinib + regorafenib + PD-1 inhibitor in a metastatic BRAF V600EMT colon cancer patient with acquired C-MET amplification and TPM4-ALK fusion: a case report.\nBackground: Colorectal cancer (CRC) with the Raf murine sarcoma viral oncogene homolog B (BRAF) V600E had a relatively poor prognosis. Anaplastic lymphoma kinase (ALK) fusion and the mesenchymal-to-epithelial transition factor (MET) amplification have been recognized as potentially important therapeutic targets in non-small cell lung cancer (NSCLC). However, both of them are of extremely lower frequencies (<2%) in metastatic CRC, and few studies have mentioned the real application of their inhibitors in CRC treatment. Case Description: A 49-year-old Chinese male was diagnosed with ascending colon adenocarcinoma (cT3N+?M1) with liver metastases. The patient performed next-generation sequencing (NGS) using tissue and circulating tumor DNA (ctDNA), and the results showed a BRAF V600E mutation. He received an

In [280]:

api = StudyCharacteristicsExtraction()
extracted = api.run(
    papers=papers0,
    fields=[f'Crizotinib Results, string, the effectiveness of treating {fin_condition} with {treatements_eng[0]}',
           'Participants, string, number of participants'],#api.DEFAULT_FIELDS,
    llm=os.getenv("MODEL_NAME"),
    chunk_size=500,
    chunk_overlap=200,
    thinking=False,
)
extracted

You are now the following python function: ```
def extract_fields_from_input_study(inputs: Dict[str, Any]) -> str:
    """
    This function is tasked with analyzing clinical trial study reports or papers to extract specific information as structured data and provide citations for the extracted information.
    The user will provide a list of fields they are interested in, along with a natural language description for each field to guide you on what content to look for and from which parts of the report to extract it.

    IMPORTANT:
    For each field described by the user, you need to:
    1. Identify and extract the relevant information from the report, based on the provided description.
    2. Generate a field name that accurately represents the content of the field based on its description.
    3. Structure the extracted information into a standard format whenever possible (e.g., integer, numerical values, dates, keywords, list of terms). 
        If standardization is not possibl

[[{'name': 'Crizotinib_Results',
   'value': 'Administration of crizotinib combined with regorafenib and tislelizumab obtained an obvious response.',
   'source_id': [3],
   'cited_blocks': ['reported a co-existence of a BRAF V600E mutation, c-MET amplification and TPM4-ALK fusion in a CRC patient. Administration of crizotinib combined with regorafenib and tislelizumab obtained an obvious response. Furthermore, continuous ctDNA detection appears to be a promising technique to monitor tumor burden, which may provide better clinical decision support during the disease course.']},
  {'name': 'Participants',
   'value': 'NP',
   'source_id': [],
   'cited_blocks': []}],
 [{'name': 'Crizotinib_Results',
   'value': 'The best clinical response was stable disease in seven patients (29%) when treated with crizotinib.',
   'source_id': [4],
   'cited_blocks': ['B.D, days 1-21 every 28 days. One in six patients exhibited a dose-limiting toxicity at this dose level. Drug-related adverse events we

In [281]:
extracted_gm = extracted
extracted_gm

[[{'name': 'Crizotinib_Results',
   'value': 'Administration of crizotinib combined with regorafenib and tislelizumab obtained an obvious response.',
   'source_id': [3],
   'cited_blocks': ['reported a co-existence of a BRAF V600E mutation, c-MET amplification and TPM4-ALK fusion in a CRC patient. Administration of crizotinib combined with regorafenib and tislelizumab obtained an obvious response. Furthermore, continuous ctDNA detection appears to be a promising technique to monitor tumor burden, which may provide better clinical decision support during the disease course.']},
  {'name': 'Participants',
   'value': 'NP',
   'source_id': [],
   'cited_blocks': []}],
 [{'name': 'Crizotinib_Results',
   'value': 'The best clinical response was stable disease in seven patients (29%) when treated with crizotinib.',
   'source_id': [4],
   'cited_blocks': ['B.D, days 1-21 every 28 days. One in six patients exhibited a dose-limiting toxicity at this dose level. Drug-related adverse events we

In [255]:

api = StudyCharacteristicsExtraction()
extracted = api.run(
    papers=papers0,
    fields=[f'Crizotinib Results, string, the effectiveness of treating {fin_condition} with {treatements_eng[0]}',
           'Participants, string, number of participants'],#api.DEFAULT_FIELDS,
    llm=os.getenv("MODEL_NAME"),
    chunk_size=500,
    chunk_overlap=200,
    thinking=False,
)
extracted

You are now the following python function: ```
def extract_fields_from_input_study(inputs: Dict[str, Any]) -> str:
    """
    This function is tasked with analyzing clinical trial study reports or papers to extract specific information as structured data and provide citations for the extracted information.
    The user will provide a list of fields they are interested in, along with a natural language description for each field to guide you on what content to look for and from which parts of the report to extract it.

    IMPORTANT:
    For each field described by the user, you need to:
    1. Identify and extract the relevant information from the report, based on the provided description.
    2. Generate a field name that accurately represents the content of the field based on its description.
    3. Structure the extracted information into a standard format whenever possible (e.g., integer, numerical values, dates, keywords, list of terms). 
        If standardization is not possibl

RetryError: RetryError[<Future at 0x1a2d1f324d0 state=finished raised APIStatusError>]

### with hf

In [269]:
os.environ["BASE_URL"] = "https://router.huggingface.co/v1"
os.environ["MODEL_NAME"] ='zai-org/GLM-4.7'#'meta-llama/Llama-3.3-70B-Instruct'
os.environ["OPENAI_API_KEY"] = os.getenv("HUGGINGFACE_HUB_TOKEN2")

In [271]:

api = StudyCharacteristicsExtraction()
extracted = api.run(
    papers=papers0[:2],
    fields=[f'Crizotinib Results, string, the effectiveness of treating {fin_condition} with {treatements_eng[0]}',
           'Participants, string, number of participants'],#api.DEFAULT_FIELDS,
    llm=os.getenv("MODEL_NAME"),
    chunk_size=500,
    chunk_overlap=200,
    thinking=False,
)
extracted

You are now the following python function: ```
def extract_fields_from_input_study(inputs: Dict[str, Any]) -> str:
    """
    This function is tasked with analyzing clinical trial study reports or papers to extract specific information as structured data and provide citations for the extracted information.
    The user will provide a list of fields they are interested in, along with a natural language description for each field to guide you on what content to look for and from which parts of the report to extract it.

    IMPORTANT:
    For each field described by the user, you need to:
    1. Identify and extract the relevant information from the report, based on the provided description.
    2. Generate a field name that accurately represents the content of the field based on its description.
    3. Structure the extracted information into a standard format whenever possible (e.g., integer, numerical values, dates, keywords, list of terms). 
        If standardization is not possibl

[[{'name': 'Crizotinib Results',
   'value': 'Partial response (PR) was achieved when crizotinib was combined with regorafenib and tislelizumab. The treatment resulted in significantly reduced BRAF mutation frequency, and MET amplification and TPM4-ALK fusion became undetectable. The patient achieved 11 months overall survival.',
   'source_id': [3, 4, 8],
   'cited_blocks': ['reported a co-existence of a BRAF V600E mutation, c-MET amplification and TPM4-ALK fusion in a CRC patient. Administration of crizotinib combined with regorafenib and tislelizumab obtained an obvious response. Furthermore, continuous ctDNA detection appears to be a promising technique to monitor tumor burden, which may provide better clinical decision support during the disease course.',
    'Partial response to crizotinib + regorafenib + PD-1 inhibitor in a metastatic BRAF V600EMT colon cancer patient with acquired C-MET amplification and TPM4-ALK fusion: a case report.',
    '(tislelizumab, dabrafenib, and tram

In [272]:
1

1

In [273]:
extracted

[[{'name': 'Crizotinib Results',
   'value': 'Partial response (PR) was achieved when crizotinib was combined with regorafenib and tislelizumab. The treatment resulted in significantly reduced BRAF mutation frequency, and MET amplification and TPM4-ALK fusion became undetectable. The patient achieved 11 months overall survival.',
   'source_id': [3, 4, 8],
   'cited_blocks': ['reported a co-existence of a BRAF V600E mutation, c-MET amplification and TPM4-ALK fusion in a CRC patient. Administration of crizotinib combined with regorafenib and tislelizumab obtained an obvious response. Furthermore, continuous ctDNA detection appears to be a promising technique to monitor tumor burden, which may provide better clinical decision support during the disease course.',
    'Partial response to crizotinib + regorafenib + PD-1 inhibitor in a metastatic BRAF V600EMT colon cancer patient with acquired C-MET amplification and TPM4-ALK fusion: a case report.',
    '(tislelizumab, dabrafenib, and tram

### all in one 

In [121]:
fin_condition, treatements_eng[0]

('Colorectal cancer', 'Crizotinib')

In [122]:
ecs['title_criteria']+ecs['content_criteria']

["Does the title mention 'colorectal cancer'?",
 "Does the title mention 'crizotinib'?",
 'Is there evidence of randomization or control group in the abstract?',
 'Are outcomes related to tumor response, progression-free survival, overall survival, or adverse events mentioned in the abstract?']

In [123]:
all_studies['screening'] = ec_pred
#[all_studies.screen_eval>=0]
all_studies

,hasResults,nctId,briefTitle,overallStatus,briefSummary,conditions,studyType,phases,interventions,outcomeMeasures,screen_eval,screening
0,False,NCT03792568,ALK Inhibitor in Metastatic Colorectal Cancer ...,UNKNOWN,This clinical trial aims to evaluate the effic...,[Colorectal Cancer],INTERVENTIONAL,[NA],[],NaN,3,"{'evaluations': ['YES', 'YES', 'UNCERTAIN', 'Y..."
1,False,NCT02568267,Basket Study of Entrectinib (RXDX-101) for the...,ACTIVE_NOT_RECRUITING,"This is an open-label, multicenter, global Pha...","[Breast Cancer, Cholangiocarcinoma, Colorectal...",INTERVENTIONAL,[PHASE2],[entrectinib],NaN,-3,"{'evaluations': ['NO', 'NO', 'UNCERTAIN', 'NO'..."
2,True,NCT02510001,MEK and MET Inhibition in Colorectal Cancer,COMPLETED,This trial is designed to try two new cancer d...,"[Solid Tumor, Colorectal Cancer]",INTERVENTIONAL,[PHASE1],"[pf-02341066, pd-0325901, binimetinib]","[{'type': 'PRIMARY', 'title': 'Maximal Tolerat...",1,"{'evaluations': ['YES', 'NO', 'UNCERTAIN', 'YE..."
3,False,NCT03947385,Study of IDE196 in Patients With Solid Tumors ...,RECRUITING,"This is a Phase 1/2, multi-center, open-label ...","[Metastatic Uveal Melanoma, Cutaneous Melanoma...",INTERVENTIONAL,"[PHASE1, PHASE2]","[ide196, binimetinib, crizotinib]",NaN,0,"{'evaluations': ['NO', 'YES', 'UNCERTAIN', 'UN..."
4,False,NCT06214793,Taletrectinib in Previously Treated Metastatic...,SUSPENDED,The purpose of this study is to evaluate the s...,"[Breast Cancer, Metastatic Breast Cancer]",INTERVENTIONAL,[PHASE2],[taletrectinib],NaN,-3,"{'evaluations': ['NO', 'NO', 'UNCERTAIN', 'NO'..."
5,False,NCT05725200,Study to Investigate Outcome of Individualized...,RECRUITING,The purpose of the study is to investigate the...,[Metastatic Colorectal Cancer],INTERVENTIONAL,[PHASE2],"[alectinib, cetuximab, crizotinib, dasatinib, ...",NaN,3,"{'evaluations': ['YES', 'YES', 'UNCERTAIN', 'Y..."
6,False,NCT02465060,Targeted Therapy Directed by Genetic Testing i...,ACTIVE_NOT_RECRUITING,This phase II MATCH screening and multi-sub-tr...,"[Advanced Lymphoma, Advanced Malignant Solid N...",INTERVENTIONAL,[PHASE2],"[adavosertib, afatinib, afatinib dimaleate, bi...",NaN,0,"{'evaluations': ['NO', 'YES', 'UNCERTAIN', 'UN..."


In [124]:
resultsct

[]

In [136]:
["Does the content mention 'colorectal cancer'?",
                 "Does content mention 'Crizotinib'?", 
                'Is there a description of treatement results?']

["Does the content mention 'colorectal cancer'?",
 "Does content mention 'Crizotinib'?",
 'Is there a description of treatement results?']

In [207]:
len(extracted)

18

In [456]:
df_papers[0].loc[df_papers[0].screen_eval>=total_ev, 'res_extr'].shape

(13,)

In [457]:
df_papers[0]['screening'] = ec_predP
df_papers[0]['res_extr'] = ''


In [458]:
df_papers[0].head(4)

,PMID,Journal,Year,Month,Day,Title,Publication Type,Authors,Abstract,screen_eval,FullText,screening,res_extr
0,40526090,Oncotarget,2025,Jun,17,Case Report WIN-MTB-2023001 WIN International ...,"Journal Article, Case Reports","Alberto Hernando-Calvo, Razelle Kurzrock, Nadi...",Heavily pretreated metastatic colorectal cance...,2,,"{'evaluations': ['YES', 'YES', 'UNCERTAIN'], '...",
1,38711893,AME case reports,2024,,,Partial response to crizotinib + regorafenib +...,"Case Reports, Journal Article","Yingying Huang, Shuai Zhang, Xueqing Hu, Xiang...",Colorectal cancer (CRC) with the Raf murine sa...,3,Partial response to crizotinib + regorafenib +...,"{'evaluations': ['YES', 'YES', 'YES'], 'ration...",
2,40140597,BJC reports,2025,Mar,26,A phase Ia study of the MEK1/2 inhibitor PD-03...,Journal Article,"Peter Gallagher, Christian Rolfo, Elena Elez, ...",Single-agent MEK1/2 inhibition has been disapp...,3,A phase Ia study of the MEK1/2 inhibitor PD-03...,"{'evaluations': ['YES', 'YES', 'YES'], 'ration...",
3,35343863,Expert opinion on therapeutic patents,2022,Jun,,An updated patent review of small-molecule ROS...,"Journal Article, Review","Meng Liu, Jintian Dai, Mudan Wei, Qingshan Pan...",C-ros oncogene 1 (ROS1) is the sole member of ...,2,,"{'evaluations': ['YES', 'YES', 'UNCERTAIN'], '...",


In [459]:
df_papers[0].head(4).screening.values

array([{'evaluations': ['YES', 'YES', 'UNCERTAIN'], 'rationale': ["The paper mentions 'metastatic colorectal cancer', which implies colorectal cancer.", "The term 'Crizotinib' is explicitly mentioned in the title and abstract.", 'There is no detailed description of treatment outcomes; only brief mention of treatment.']},
       {'evaluations': ['YES', 'YES', 'YES'], 'rationale': ["The paper mentions 'ascending colon adenocarcinoma', which is part of colorectal cancer.", "The term 'Crizotinib' is explicitly mentioned in the title and throughout the abstract and body of the paper.", 'The paper describes treatment outcomes including partial response to Crizotinib, regorafenib, and PD-1 inhibitor.']},
       {'evaluations': ['YES', 'YES', 'YES'], 'rationale': ["The paper mentions 'advanced solid cancers', but explicitly states 'Patients with Colorectal cancer undergoing treatment with Crizotinib'", "The intervention described is 'Crizotinib'", "The paper reports 'best clinical response was

In [550]:
all_studies[(all_studies.screen_eval>=0)&(all_studies.hasResults)]

,hasResults,nctId,briefTitle,overallStatus,briefSummary,conditions,studyType,phases,interventions,outcomeMeasures,screen_eval,screening
2,True,NCT02510001,MEK and MET Inhibition in Colorectal Cancer,COMPLETED,This trial is designed to try two new cancer d...,"[Solid Tumor, Colorectal Cancer]",INTERVENTIONAL,[PHASE1],"[pf-02341066, pd-0325901, binimetinib]","[{'type': 'PRIMARY', 'title': 'Maximal Tolerat...",1,"{'evaluations': ['YES', 'NO', 'UNCERTAIN', 'YE..."


In [551]:
resultsct

['```json\n[\n    {\n        "population": 30,\n        "time_frame": "Dose Expansion phase: change from baseline and up to 12 months.",\n        "outcomes": [\n            {\n                "category_name": "Stable Disease",\n                "outcome": 7\n            },\n            {\n                "category_name": "Progressive Disease",\n                "outcome": 22\n            },\n            {\n                "category_name": "Early death from malignant disease",\n                "outcome": 1\n            }\n        ]\n    }\n]\n```']

In [558]:
to_work#.res_with_part#.values

,hasResults,nctId,briefTitle,overallStatus,briefSummary,conditions,studyType,phases,interventions,outcomeMeasures,screen_eval,screening,res_with_part
2,True,NCT02510001,MEK and MET Inhibition in Colorectal Cancer,COMPLETED,This trial is designed to try two new cancer d...,"[Solid Tumor, Colorectal Cancer]",INTERVENTIONAL,[PHASE1],"[pf-02341066, pd-0325901, binimetinib]","[{'type': 'PRIMARY', 'title': 'Maximal Tolerat...",1,"{'evaluations': ['YES', 'NO', 'UNCERTAIN', 'YE...","[{'type': 'PRIMARY', 'title': 'Clinical Respon..."


In [559]:

ids_ct = to_work#all_studies[(all_studies.screen_eval>=0)&(all_studies.hasResults)]#.nctId.values
ids_p = df_papers[0][df_papers[0].screen_eval>=total_ev]#.PMID.values

In [462]:
all_studies[(all_studies.screen_eval>=0)&(all_studies.hasResults)]

,hasResults,nctId,briefTitle,overallStatus,briefSummary,conditions,studyType,phases,interventions,outcomeMeasures,screen_eval,screening
2,True,NCT02510001,MEK and MET Inhibition in Colorectal Cancer,COMPLETED,This trial is designed to try two new cancer d...,"[Solid Tumor, Colorectal Cancer]",INTERVENTIONAL,[PHASE1],"[pf-02341066, pd-0325901, binimetinib]","[{'type': 'PRIMARY', 'title': 'Maximal Tolerat...",1,"{'evaluations': ['YES', 'NO', 'UNCERTAIN', 'YE..."


In [178]:
all_studies[(all_studies.screen_eval>=0)&(all_studies.hasResults)].values.shape

(1, 12)

In [193]:
df_papers[0].head(1)

,PMID,Journal,Year,Month,Day,Title,Publication Type,Authors,Abstract,screen_eval,FullText,screening,res_extr
0,40526090,Oncotarget,2025,Jun,17,Case Report WIN-MTB-2023001 WIN International ...,"Journal Article, Case Reports","Alberto Hernando-Calvo, Razelle Kurzrock, Nadi...",Heavily pretreated metastatic colorectal cance...,2,Case Report WIN-MTB-2023001 WIN International ...,"{'evaluations': ['YES', 'YES', 'UNCERTAIN'], '...",


In [302]:
import itertools
a = [i['value'] for i in gg[:,0] if i['value']!='NP' ]
#list(itertools.chain.from_iterable(a))
len(' '.join(a))

1344

In [549]:
resultsct

['```json\n[\n    {\n        "population": 30,\n        "time_frame": "Dose Expansion phase: change from baseline and up to 12 months.",\n        "outcomes": [\n            {\n                "category_name": "Stable Disease",\n                "outcome": 7\n            },\n            {\n                "category_name": "Progressive Disease",\n                "outcome": 22\n            },\n            {\n                "category_name": "Early death from malignant disease",\n                "outcome": 1\n            }\n        ]\n    }\n]\n```']

In [463]:
print(f'Condition: {fin_condition} \n\nTreatement 1: {treatements_eng[0]}')
print(f'\nChosen clinical trials:')
for i, ext_res in zip(ids_ct.values,resultsct):
    print('- https://clinicaltrials.gov/study/'+i[1]+' '+f"'{i[2]}'")
    print(ext_res)
print(f'\nChosen Papers:')
for i, ext_res in zip(ids_p.values, extracted_gm):
    print('- https://pubmed.ncbi.nlm.nih.gov/'+i[0]+' '+f"'{i[5]}'")
    for field in ext_res:
        if field['source_id']:
            print(f"    {field['name'].replace('_',' ')}: '{field['value']}'")
            print(field['cited_blocks'])

Condition: Colorectal cancer 

Treatement 1: Crizotinib

Chosen clinical trials:
- https://clinicaltrials.gov/study/NCT02510001 'MEK and MET Inhibition in Colorectal Cancer'
For the PRIMARY outcome measure "Clinical Response to Binimetinib Combined With PF-02341066":

- **Population Description**: Evaluable patients include those completing a response assessment post-cycle 1 or experiencing early progression.
- **Reporting Status**: POSTED
- **Parameter Type**: COUNT_OF_PARTICIPANTS
- **Unit of Measure**: Participants
- **Time Frame**: Dose expansion phase - changes observed within 12 months

**Efficiency Statistics**:
- Total participants evaluated: 30
- Outcomes:
  - Stable Disease: 7 participants
  - Progressive Disease: 22 participants
  - Early Death from Malignant Disease: 1 participa

Chosen Papers:
- https://pubmed.ncbi.nlm.nih.gov/38711893 'Partial response to crizotinib + regorafenib + PD-1 inhibitor in a metastatic'
    Crizotinib Results: 'Administration of crizotinib combi

In [467]:
df_papers[0][df_papers[0].screen_eval>=3]

,PMID,Journal,Year,Month,Day,Title,Publication Type,Authors,Abstract,screen_eval,FullText,screening,res_extr
1,38711893,AME case reports,2024,,,Partial response to crizotinib + regorafenib +...,"Case Reports, Journal Article","Yingying Huang, Shuai Zhang, Xueqing Hu, Xiang...",Colorectal cancer (CRC) with the Raf murine sa...,3,Partial response to crizotinib + regorafenib +...,"{'evaluations': ['YES', 'YES', 'YES'], 'ration...",
2,40140597,BJC reports,2025,Mar,26,A phase Ia study of the MEK1/2 inhibitor PD-03...,Journal Article,"Peter Gallagher, Christian Rolfo, Elena Elez, ...",Single-agent MEK1/2 inhibition has been disapp...,3,A phase Ia study of the MEK1/2 inhibitor PD-03...,"{'evaluations': ['YES', 'YES', 'YES'], 'ration...",
4,41213063,"Journal of environmental pathology, toxicology...",2025,,,Radiotherapy-Related Autophagy Genes Predict P...,Journal Article,"Mao Mao, Yichao Zhuang, Yan Chen",Numerous genes have been associated with color...,3,Radiotherapy-Related Autophagy Genes Predict P...,"{'evaluations': ['YES', 'YES', 'YES'], 'ration...",
6,34898475,Aging,2021,Dec,13,Identification of a novel immune signature for...,"Journal Article, Research Support, Non-U.S. Gov't","Yan Li, Yiyi Li, Zijin Xia, Dun Zhang, Xiaomei...","Globally, colorectal cancer (CRC) is one of th...",3,Identification of a novel immune signature for...,"{'evaluations': ['YES', 'YES', 'YES'], 'ration...",
8,37418240,ACS applied materials & interfaces,2023,Jul,19,Albumin-Based Cyanine Crizotinib Conjugate Nan...,Journal Article,"Zhuang Hu, Ruihan Li, Xinyue Cui, Zilin Chen",Colorectal cancer (CRC) is presently the third...,3,Albumin-Based Cyanine Crizotinib Conjugate Nan...,"{'evaluations': ['YES', 'YES', 'YES'], 'ration...",
9,36316649,BMC cancer,2022,Oct,31,Personalized targeted therapy prescription in ...,Journal Article,"Maxim Sorokin, Marianna Zolotovskaia, Daniil N...",Overall survival of advanced colorectal cancer...,3,Personalized targeted therapy prescription in ...,"{'evaluations': ['YES', 'YES', 'YES'], 'ration...",
10,40369167,Cell death and differentiation,2025,Nov,,ARID1A loss enhances sensitivity to c-MET inhi...,Journal Article,"Xu Zhang, Zihuan Wang, Yilin He, Kejin Wang, C...","ARID1A, a subunit of the SWI/SNF chromatin-rem...",3,ARID1A loss enhances sensitivity to c-MET inhi...,"{'evaluations': ['YES', 'YES', 'YES'], 'ration...",
11,36053834,International journal of cancer,2022,Dec,15,ROS1 genomic rearrangements are rare actionabl...,Journal Article,"Dilara Akhoundova, Saskia Hussung, Smruthy Siv...","c-Ros oncogene 1, receptor tyrosine kinase (RO...",3,ROS1 genomic rearrangements are rare actionabl...,"{'evaluations': ['YES', 'YES', 'YES'], 'ration...",
13,40211189,BMC cancer,2025,Apr,10,A Phase Ia/b study of MEK1/2 inhibitor binimet...,"Journal Article, Clinical Trial, Phase I","Francesca Aroldi, Elena Elez, Thierry André, G...",Targeting RAS mutant (MT) colorectal cancer (C...,3,A Phase Ia/b study of MEK1/2 inhibitor binimet...,"{'evaluations': ['YES', 'YES', 'YES'], 'ration...",
14,38212428,Cellular and molecular life sciences : CMLS,2024,Jan,12,Inhibition of autocrine HGF maturation overcom...,Journal Article,"Vivian Truong Jones, Ramona Graves-Deal, Zheng...",Although amplifications and mutations in recep...,3,Inhibition of autocrine HGF maturation overcom...,"{'evaluations': ['YES', 'YES', 'YES'], 'ration...",


In [407]:
ids_ct.values[0][3]

'COMPLETED'

In [421]:
extracted_gm[10]

[{'name': 'Crizotinib_Results',
  'value': 'NP',
  'source_id': [],
  'cited_blocks': []},
 {'name': 'Participants', 'value': 'NP', 'source_id': [], 'cited_blocks': []}]

In [528]:
json_resultsct = parse_json_outputs(resultsct)
json_resultsct

[[{'population': 30,
   'time_frame': 'Dose Expansion phase: change from baseline and up to 12 months.',
   'outcomes': [{'category_name': 'Stable Disease', 'outcome': 7},
    {'category_name': 'Progressive Disease', 'outcome': 22},
    {'category_name': 'Early death from malignant disease', 'outcome': 1}]}]]

In [544]:
to_work.res_with_part.values[0]

[{'type': 'PRIMARY',
  'title': 'Clinical Response to Binimetinib Combined With PF-02341066',
  'description': 'To investigate response to treatment with RPII dose of Binimetinib with Crizotinib (PF-02341066), in patients with a) RASMT CRC or b) RASWT/cMET mut amplified CRC or c) RASWT/c-MET over-expressed CRC, as defined by stable, partially or completely responding disease, per Response Evaluation Criteria In Solid Tumors Criteria (RECIST v1.1) for target lesions and assessed by CT: Complete Response (CR), Disappearance of all target lesions; Partial Response (PR), \\>=30% decrease in the sum of the longest diameter of target lesions; Stable Disease (SD), Neither sufficient shrinkage to qualify for PR nor sufficient increase (\\>=20%) to qualify for Progressive Disease; Overall Response (OR) = CR + PR + SD',
  'populationDescription': 'Evaluable patients for the primary outcome are those patients who complete a response assessment after cycle 1 of treatment, or who progress early on 

In [548]:
ids_ct.values[0][9]

[{'type': 'PRIMARY',
  'title': 'Maximal Tolerated Dose (MTD) of PD-0325901 and PF-02341066 /PF-02341066 or Binimetinib With PF-02341066',
  'description': 'Determine maximum tolerated dose (MTD) of PD-0325901 with Crizotinib (PF-02341066) according to toxicities graded by NCI CTCAE v4.03, in patients with advanced solid tumours.',
  'populationDescription': 'Evaluable patients are those patients who completed cycle 1 or withdrew early for experiencing a DLT. Four patients withdrew early from the study not for DLTs, consequently 21 patients remained for the dose escalation primary analysis.',
  'reportingStatus': 'POSTED',
  'paramType': 'NUMBER',
  'unitOfMeasure': 'Dose Limiting Toxicities (DLTs)',
  'timeFrame': 'Dose Escalation Phase: treatment Cycle 1 28 days (plus 7 day run-in for PD-0325901/PF-02341066 combination)',
  'groups': [{'id': 'OG000',
    'title': 'Dose Escalation Phase Cohort 1 Dose Level 1',
    'description': 'Crizotinib (PF-02341066) 250mg OD Days 1-28 continuousl

In [545]:
json_resultsct

[[{'population': 30,
   'time_frame': 'Dose Expansion phase: change from baseline and up to 12 months.',
   'outcomes': [{'category_name': 'Stable Disease', 'outcome': 7},
    {'category_name': 'Progressive Disease', 'outcome': 22},
    {'category_name': 'Early death from malignant disease', 'outcome': 1}]}]]

In [567]:
to_work.values[0][-1]

[{'type': 'PRIMARY',
  'title': 'Clinical Response to Binimetinib Combined With PF-02341066',
  'description': 'To investigate response to treatment with RPII dose of Binimetinib with Crizotinib (PF-02341066), in patients with a) RASMT CRC or b) RASWT/cMET mut amplified CRC or c) RASWT/c-MET over-expressed CRC, as defined by stable, partially or completely responding disease, per Response Evaluation Criteria In Solid Tumors Criteria (RECIST v1.1) for target lesions and assessed by CT: Complete Response (CR), Disappearance of all target lesions; Partial Response (PR), \\>=30% decrease in the sum of the longest diameter of target lesions; Stable Disease (SD), Neither sufficient shrinkage to qualify for PR nor sufficient increase (\\>=20%) to qualify for Progressive Disease; Overall Response (OR) = CR + PR + SD',
  'populationDescription': 'Evaluable patients for the primary outcome are those patients who complete a response assessment after cycle 1 of treatment, or who progress early on 

In [572]:
all_studies[all_studies.screen_eval>=0]['outcomeMeasures'].values[1]

[{'type': 'PRIMARY',
  'title': 'Maximal Tolerated Dose (MTD) of PD-0325901 and PF-02341066 /PF-02341066 or Binimetinib With PF-02341066',
  'description': 'Determine maximum tolerated dose (MTD) of PD-0325901 with Crizotinib (PF-02341066) according to toxicities graded by NCI CTCAE v4.03, in patients with advanced solid tumours.',
  'populationDescription': 'Evaluable patients are those patients who completed cycle 1 or withdrew early for experiencing a DLT. Four patients withdrew early from the study not for DLTs, consequently 21 patients remained for the dose escalation primary analysis.',
  'reportingStatus': 'POSTED',
  'paramType': 'NUMBER',
  'unitOfMeasure': 'Dose Limiting Toxicities (DLTs)',
  'timeFrame': 'Dose Escalation Phase: treatment Cycle 1 28 days (plus 7 day run-in for PD-0325901/PF-02341066 combination)',
  'groups': [{'id': 'OG000',
    'title': 'Dose Escalation Phase Cohort 1 Dose Level 1',
    'description': 'Crizotinib (PF-02341066) 250mg OD Days 1-28 continuousl

In [588]:
[i[0]['value'] for i in extracted_gm if i[0]['value']!='NP']

['Administration of crizotinib combined with regorafenib and tislelizumab obtained an obvious response.',
 'The best clinical response was stable disease in seven patients (29%) when treated with crizotinib.',
 'Drugs such as Crizotinib were more sensitive to low-risk group.',
 'AZD4547, Cytochalasin B and S-crizotinib might have potential therapeutic implications in the immune risk score-high CRCs.',
 'Under 808 nm laser irradiation, Crizotinib-IR808@BSA NPs exhibited synergistic chemophototherapy effects on tumors.',
 'Crizotinib selectively inhibits the growth of ARID1A-deficient CRC cells in vitro and in xenograft tumor models.',
 'Molecularly targeted treatment with crizotinib induced a rapid and sustained partial response. After 15 months on crizotinib disseminated tumor progression occurred and KRAS Q61H emerged in tissue and liquid biopsies.',
 'Combination binimetinib/crizotinib showed a poor tolerability with no objective responses observed in RASMT advanced CRC patients.',
 

In [601]:
SUMMARY  = f'''You are a clinical specialist conducting a clinicial meta-analysis on treating {fin_condition} with {treatements_eng[0]}. 

The user will provide a list of text sources.
Your task is to summarize results regarding Crizotinib effectiviness from several sources.

Give a short summary in 3-4 sentences.
Your answer will be shown to a doctor for desicion making, do not include any additional text.
'''
SUMMARY

'You are a clinical specialist conducting a clinicial meta-analysis on treating Colorectal cancer with Crizotinib. \n\nThe user will provide a list of text sources.\nYour task is to summarize results regarding Crizotinib effectiviness from several sources.\n\nGive a short summary in 3-4 sentences.\nYour answer will be shown to a doctor for desicion making, do not include any additional text.\n'

In [602]:
at_once = False
openai_client = OpenAI(
    base_url=os.getenv("BASE_URL"),
    api_key=os.getenv("OPENAI_API_KEY"),
    http_client=httpx.Client(verify=False)
)
ss = [i[0]['value'] for i in extracted_gm if i[0]['value']!='NP']
messages2 = [{'role':'system', 'content':SUMMARY+' \no_think'},
                {'role':'user', 'content':f"{ss}"}]
   
response2 = openai_client.chat.completions.create(
    model=os.getenv('MODEL_NAME'),
    messages=messages2,
    temperature=0,
)
fin2 = response2.choices[0].message.content
answer2 = fin2.strip('<think>\n\n</think>\n\n')
answer2


'Crizotinib demonstrates effectiveness in combination therapies for colorectal cancer, particularly enhancing responses when paired with other agents like regorafenib or tislelizumab. It shows selective inhibition against specific mutations, including ARID1A deficiency and HGF-mediated cetuximab resistance. However, its efficacy varies across different patient groups; while it can induce partial responses, some combinations exhibit poor tolerability without significant benefits. Additionally, biomarkers like SDHB levels correlate with drug sensitivity.'

In [603]:
print(answer2)

Crizotinib demonstrates effectiveness in combination therapies for colorectal cancer, particularly enhancing responses when paired with other agents like regorafenib or tislelizumab. It shows selective inhibition against specific mutations, including ARID1A deficiency and HGF-mediated cetuximab resistance. However, its efficacy varies across different patient groups; while it can induce partial responses, some combinations exhibit poor tolerability without significant benefits. Additionally, biomarkers like SDHB levels correlate with drug sensitivity.


In [606]:
import random

from pdfme import build_pdf


random.seed(1)
abc = 'abcdefghijklmnñopqrstuvwxyzABCDEFGHIJKLMNÑOPQRSTUVWXYZáéíóúÁÉÍÓÚ'

def gen_word():
    return ''.join(random.choice(abc) for _ in range(random.randint(1, 10)))

def gen_text(n):
    return random.choice(['',' ']) + (' '.join(gen_word() for _ in range(n))) + random.choice(['',' '])

def gen_paragraphs(n):
    return [gen_text(random.randint(50, 200)) for _ in range(n)]

def clin():
    func_r = []
    num = 1
    for i, ext_res in zip(ids_ct.values,json_resultsct):
        func_r.append([f"{num}. '{i[2]}'", 
                       f" {i[3].lower()}", 
                       f" {' '.join(i[7]).lower()}",
                       {".": " (link)", "style": "url", 
                            "uri": f'https://clinicaltrials.gov/study/{i[1]}'}, 
                      ])
        #func_r.append(['- https://clinicaltrials.gov/study/'+i[1]+' '+f"'{i[2]}'"])
        #func_r.append([ext_res])
        for j_n, j in enumerate(ext_res):
            func_r_1 = []
            func_r_1 = func_r_1+ ['Measure: ', i[-1][j_n]['title'], '\n',
                          {".s:8;i": f'Population: {j['population']}'}, '\n',
                          {".s:8;i": f'Time: {j['time_frame']}'}, '\n',
                          ]
            for out in j['outcomes']:
                func_r_1 = func_r_1+ [{".s:8;i": out['category_name']},
                                ': ',
                               {".s:8;i": out['outcome']}, '\n',
                              ]
            func_r.append(func_r_1)
        num+=1
    return func_r

def papers():
    func_r = []
    num = 1
    for i, ext_res in zip(ids_p.values, extracted_gm):
        func_r.append([f"{num}. '{i[5]}'", 
                       {".": f" (link)", "style": "url", 
                            "uri": f'https://pubmed.ncbi.nlm.nih.gov/{i[0]}'}, 
                      ])
        if ext_res[0]['source_id']:
            func_r.append([f"    {ext_res[0]['name'].replace('_',' ')}: '{ext_res[0]['value']}'"+
                          " Based on:"])
            for cb in ext_res[0]['cited_blocks']: 
                func_r.append([{".s:8;i":f"'{cb}'"}])
            #func_r.append(field['cited_blocks'])
        num+=1
    return func_r


document = {
    "style": {
        "margin_bottom": 15, "text_align": "j",
        "page_size": "letter", "margin": [60, 50]
    },
    "formats": {
        "url": {"c": "blue", "u": 1},
        "title": {"b": 1, "s": 13}
    },
    "running_sections": {
        "header": {
            "x": "left", "y": 20, "height": "top", "style": {"text_align": "r"},
            "content": [{".b": "This is a header"}]
        },
        "footer": {
            "x": "left", "y": 740, "height": "bottom", "style": {"text_align": "c"},
            "content": [{".": ["Page ", {"var": "$page"}]}]
        }
    },
    "sections": [
        {
            "style": {"page_numbering_style": "arabic"},
            "running_sections": ["footer"],
            "content": [

                {
                    ".": f"Condition: {fin_condition}", "style": "title", "label": "title1",
                },
                [{".b": f'Treatement 1: {treatements_eng[0]}'}],
                [{".b": 'Chosen clinical trials:'}],
                
                *clin(),
                
                [{".b": 'Chosen papers:'}],
                [f'Short summary: {answer2}'],
                *papers(),
                
                #{"image": "path/to/some/image.jpg"},

                
            ]
        }
    ]
}

with open('document.pdf', 'wb') as f:
    build_pdf(document, f)

In [ ]:
pdf = FPDF()
pdf.add_page()

pdf.set_font('Arial', '', 14)

pdf.cell(40, 10, f'Condition: {fin_condition}')

pdf.cell(40, 10, f'Treatement 1: {treatements_eng[0]}')

pdf.set_font('Arial', 'B', 12)
pdf.cell(40, 10, f'Chosen clinical trials:')

pdf.set_font('Arial', '', 12)
for i, ext_res in zip(ids_ct.values,resultsct):
    pdf.cell(40, 10, '- https://clinicaltrials.gov/study/'+i[1]+' '+f"'{i[2]}'")
    pdf.cell(40, 10, ext_res)
'''    
print(f'\nChosen Papers:')
for i, ext_res in zip(ids_p.values, extracted_gm):
    print('- https://pubmed.ncbi.nlm.nih.gov/'+i[0]+' '+f"'{i[5]}'")
    for field in ext_res:
        if field['source_id']:
            print(f"    {field['name'].replace('_',' ')}: '{field['value']}'")
            print(field['cited_blocks'])
'''            

pdf.output(f"{fin_condition.replace('_',' ')}_{treatements_eng[0]}.pdf", 'F')

In [432]:
df_papers[0][df_papers[0].screen_eval>=3].loc[8].FullText            

'Albumin-Based Cyanine Crizotinib Conjugate Nanoparticles for NIR-II Imaging-Guided Synergistic Chemophototherapy.\nColorectal cancer (CRC) is presently the third deadliest cancer in the world. This malignant cancer usually precedes the progression of precancerous lesions, and it is challenging to distinguish its nuanced morphological changes. Molecular-based near-infrared-II (NIR-II) fluorescence imaging can effectively recognize lesion targets to improve image contrast and increase early tumor detection compared with traditional wide-light screening endoscopy. c-Met has been determined to be overexpressed in advanced stages of CRC and is considered to be a potent tumor biomarker. Herein, based on the well-targeted inhibitory effect of Crizotinib on c-Met positive tumor cells, the dye IR808 was covalently combined with the drug molecule Crizotinib, resulting in the synthesis of a NIR fluorescent probe Crizotinib-IR808 targeting c-Met positive tumor cells. Then, water-insoluble Crizoti

# d

In [81]:
pmid_list = df_papers[0].PMID.values.tolist()
#['41213063',#'26451310',]

res = pmid2biocxml(pmid_list, api_key=pubmed_api_key)
#print(res)
res = [parse_bioc_xml(r) for r in res]
# transform the parsed xml into paper content
papers0 = []
for parsed in res:
    paper_content = []
    for parsed_ in parsed["passage"]:
        paper_content.append(parsed_['content'])
    paper_content = "\n".join(paper_content)
    papers0.append(paper_content)


https://www.ncbi.nlm.nih.gov/research/bionlp/RESTful/pmcoa.cgi/BioC_xml/40140597/unicode?api_key=6892c4129cef143ff92d11533848d2e0d908
https://www.ncbi.nlm.nih.gov/research/bionlp/RESTful/pmcoa.cgi/BioC_xml/40211189/unicode?api_key=6892c4129cef143ff92d11533848d2e0d908
https://www.ncbi.nlm.nih.gov/research/bionlp/RESTful/pmcoa.cgi/BioC_xml/40526090/unicode?api_key=6892c4129cef143ff92d11533848d2e0d908
https://www.ncbi.nlm.nih.gov/research/bionlp/RESTful/pmcoa.cgi/BioC_xml/41213063/unicode?api_key=6892c4129cef143ff92d11533848d2e0d908
https://www.ncbi.nlm.nih.gov/research/bionlp/RESTful/pmcoa.cgi/BioC_xml/41268440/unicode?api_key=6892c4129cef143ff92d11533848d2e0d908
https://www.ncbi.nlm.nih.gov/research/bionlp/RESTful/pmcoa.cgi/BioC_xml/38711893/unicode?api_key=6892c4129cef143ff92d11533848d2e0d908
https://www.ncbi.nlm.nih.gov/research/bionlp/RESTful/pmcoa.cgi/BioC_xml/40369167/unicode?api_key=6892c4129cef143ff92d11533848d2e0d908


In [ ]:
df_papers[0]

In [85]:
papers0[2]

'Case Report WIN-MTB-2023001 WIN International Molecular Tumor Board A 62-year-old male with metastatic colorectal cancer with 5 prior lines of treatment\nHeavily pretreated metastatic colorectal cancer (mCRC) poses significant therapeutic challenges. Advances in molecular profiling enables personalized strategies. We present a 62-year-old male with mCRC harboring BRAF, MET, APC, TP53 and NRAS alterations, following FOLFOX and FOLFIRI, dabrafenib plus panitumumab, and a BRAF inhibitor clinical trial, each leading to initial responses followed by disease progression.\nWIN Consortium International Molecular Tumor Board (MTB), included experts from institutions across 13 countries. Enrollment in suitable clinical trials was explored but limited by availability. Personalized combinations suggested included amivantamab-vmjw (anti-MET/EGFR antibody) (one-third standard dose) (for MET amplification and due to prior response to anti-EGFR antibody), trametinib, 1 mg po daily (MEK inhibitor for 

In [11]:
[len(i) for i in papers]

[1672, 1652, 27171, 2627, 46819]

In [15]:
splited = split_text_into_chunks(papers[1], chunk_size=600,
                                             chunk_overlap=300)
splited

['Case Report: HGF and NF1 mutations as putative bypass mechanisms of MET inhibitor resistance in hepatocellular carcinoma: a case study.',
 'Background: Hepatocellular carcinoma (HCC) is a highly aggressive liver cancer with poor prognosis, often associated with resistance to treatment. MET amplification has been identified as a potential therapeutic target, but resistance to MET inhibitors, such as crizotinib, remains a significant challenge. This study aims to explore the molecular mechanisms underlying resistance to MET inhibitors in MET-amplified HCC. Methods: We present a case of advanced HCC in a patient with MET amplification treated with crizotinib. After initial tumor regression, disease progression occurred. Genetic',
 'challenge. This study aims to explore the molecular mechanisms underlying resistance to MET inhibitors in MET-amplified HCC. Methods: We present a case of advanced HCC in a patient with MET amplification treated with crizotinib. After initial tumor regression

In [12]:

api = StudyCharacteristicsExtraction()
extracted = api.run(
    papers=papers,
    fields=['Results, string, main results of the study regarding Crizotinib, usually about treatment effectiveness'],#api.DEFAULT_FIELDS,
    llm='Qwen/Qwen3-32B',#"openai-gpt-4o",
    chunk_size=600,
    chunk_overlap=300,
    thinking=False,
)
extracted

False


[[{'name': 'Crizotinib treatment effectiveness',
   'value': 'initial tumor regression followed by disease progression',
   'source_id': [2],
   'cited_blocks': ['challenge. This study aims to explore the molecular mechanisms underlying resistance to MET inhibitors in MET-amplified HCC. Methods: We present a case of advanced HCC in a patient with MET amplification treated with crizotinib. After initial tumor regression, disease progression occurred. Genetic analysis using next-generation sequencing (NGS) was performed on biopsy samples taken before and after progression to identify mutations associated with resistance. Results: NGS revealed the loss of MET amplification and identified HGF and NF1 mutations as potential bypass mechanisms. Specifically,']}],
 [{'name': 'Crizotinib effectiveness',
   'value': 'More sensitive to low-risk group',
   'source_id': [5],
   'cited_blocks': ['performed in risk groups. Expression of signature genes in CRC cells was examined using quantitative rev

[[{'name': 'Crizotinib effectiveness',
   'value': 'More sensitive to low-risk group',
   'source_id': [4],
   'cited_blocks': ['performed in risk groups. Expression of signature genes in CRC cells was examined using quantitative reverse transcription polymerase chain reaction (qRT-PCR). There were 10 CRC-related RRAGs found. Prognostic models were built with RRAGs. Based on immune infiltration analysis, low-risk populations showed significantly greater levels of immune infiltration (P P ARHGEF17 mutation rate was 6%. Medications such as CGP-082996, Dasatinib, Erlotinib, and Salubrinal were more sensitive to high-risk group, whereas drugs such as FTI-277, DMOG, and Crizotinib were more sensitive to low-risk group.']}],
 [{'name': 'Crizotinib treatment effectiveness',
   'value': 'initial tumor regression followed by disease progression',
   'source_id': [1],
   'cited_blocks': ['challenge. This study aims to explore the molecular mechanisms underlying resistance to MET inhibitors in ME

In [38]:
llm_reasoning = ChatOpenAI(
    base_url=BASE_URL,
    api_key=HUGGINGFACE_HUB_TOKEN,
    model=MODEL_NAME_HF,
    streaming=True,
    temperature=0,
    #http_client=httpx_client
    http_async_client=httpx.AsyncClient(verify=False)
)

In [47]:
prompt = ChatPromptTemplate.from_messages([
    ("user", '/no_think {text}'),
    # ("ai", "Good.")
])

chain = prompt | llm_reasoning
response = await chain.ainvoke({"text": ff})

#fin_condition = response.content.strip('<think>\n\n</think>\n\n')
print(response.content)

<think>

</think>

```json
[
    {
        "name": "Crizotinib effectiveness",
        "value": "More sensitive to low-risk group",
        "source_id": [3]
    }
]
```
